In [21]:
import datetime
import pandas as pd
import requests
import os, json
import pyshark
from graphviz import Digraph
import graphviz as gv



Author: Aaron Weathersby
Co-Author: Mark Washington
IT820- Replication Study
Study Replicated: Extracting Attack Narratives from Traffic Datasets by Jose David Mireles, Jin-Hee Cho, Shouhuai Xu

**Readme**
**Summary of Process**
1. Wget PCAP files as .PCAP.GZ
- Using Wget statement from cyberimpact, download all PCAP files
2. Unzip PCAP files to .PCAP
- Using Excel, run Unzip command
3. Submit PCAP files to SNORT return JSON files containing alerts
- Using Excel, run script containing line by line pcap submital


4. Injest Alert JSON files into Jupyter Dataframe
5. Injest PCAPJSON into Juptyer DataFrame
6. Statitical Analysis of ALert Files
7. Map CKC Dictionary onto Alert DataFrame
8. Graph Alert Files based on common destinations (alerts, IPs, CKC)


In [22]:
#Dictionary of CKC Mapped to Snort Rules
#References: 
#https://www.lockheedmartin.com/content/dam/lockheed-martin/rms/documents/cyber/Seven_Ways_to_Apply_the_Cyber_Kill_Chain_with_a_Threat_Intelligence_Platform.pdf
#https://www.eunis.org/download/TNC2017/TNC17-IreneuszTarnowski-cybersecurity.pdf
#Note these the mapping was only done for those rules that were alerted on from the dataset
ckc_mapping={
 
 'BAD-TRAFFIC loopback traffic':'Weaponization',
'BAD-TRAFFIC same SRC/DST':'Exploitation',
'COMMUNITY WEB-MISC Ezupload Pro form.php remote file include':'Exploitation',
'COMMUNITY WEB-MISC JBoss jmx-console html adaptor access':'Exploitation',
'COMMUNITY WEB-MISC Proxy Server Access':'Exploitation',
'COMMUNITY WEB-MISC Wikiwig wk_lang.php remote file include':'Exploitation',
'COMMUNITY WEB-MISC mod_jrun overflow attempt':'Exploitation',
'COMMUNITY WEB-MISC tinyBB footers.php remote file include':'Exploitation',
'COMMUNITY WEB-PHP BosClassifieds account.php remote file include':'Exploitation',
'COMMUNITY WEB-PHP Kamikaze-QSCM config.inc access':'Exploitation',
'COMMUNITY WEB-PHP Project Eros BBSEngine comment.php access':'Exploitation',
'COMMUNITY WEB-PHP Somery team.php remote file include':'Exploitation',
'COMMUNITY WEB-PHP UBB.threads remote file include':'Exploitation',
'COMMUNITY WEB-PHP XSS attempt':'Exploitation',
'COMMUNITY WEB-PHP phpinfo access':'Exploitation',
'DNS zone transfer TCP':'Reconaissance',
'FINGER . query':'Reconaissance',
'FINGER / execution attempt':'Reconaissance',
'FINGER 0 query':'Reconaissance',
'FINGER account enumeration attempt':'Reconaissance',
'FINGER redirection attempt':'Reconaissance',
'FINGER remote command pipe execution attempt':'Weaponization',
'FINGER root query':'Reconaissance',
'FINGER search query':'Reconaissance',
'ICMP Destination Unreachable Communication Administratively Prohibited':'Reconaissance',
'ICMP Destination Unreachable Communication with Destination Host is Administratively Prohibited':'Reconaissance',
'ICMP Destination Unreachable Host Unreachable':'Reconaissance',
'ICMP Destination Unreachable Port Unreachable':'Reconaissance',
'ICMP Echo Reply':'Reconaissance',
'ICMP PING':'Reconaissance',
'ICMP PING *NIX':'Reconaissance',
'ICMP PING NMAP':'Reconaissance',
'ICMP PING Windows':'Reconaissance',
'ICMP PING undefined code':'Reconaissance',
'ICMP Time-To-Live Exceeded in Transit':'Reconaissance',
'ICMP Timestamp Request':'Reconaissance',
'ICMP traceroute':'Reconaissance',
'INFO FTP Bad login':'Exploitation',
'INFO TELNET access':'Exploitation',
'INFO web bug 0x0 gif attempt':'Delivery',
'MISC MS Terminal server request':'Delivery',
'MISC MS Terminal server request RDP':'Delivery',
'MISC xdmcp info query':'Reconaissance',
'MS-SQL ping attempt':'Reconaissance',
'NETBIOS SMB-DS IPC$ share access':'Exploitation',
'NETBIOS SMB-DS IPC$ unicode share access':'Exploitation',
'NETBIOS SMB-DS Session Setup AndX request unicode username overflow attempt':'Exploitation',
'NETBIOS SMB-DS repeated logon failure':'Delivery',
'POLICY FTP anonymous login attempt':'Delivery',
'RPC portmap snmpXdmi request TCP':'Delivery',
'RPC portmap yppasswd request TCP':'Delivery',
'RPC portmap ypupdated request TCP':'Delivery',
'RSERVICES rlogin root':'Exploitation',
'SCAN Amanda client version request':'Reconaissance',
'SCAN UPnP service discover attempt':'Reconaissance',
'SCAN nmap XMAS':'Reconaissance',
'SNMP AgentX/tcp request':'Delivery',
'SNMP missing community string attempt':'Delivery',
'SNMP private access udp':'Exploitation',
'SNMP public access udp':'Reconaissance',
'SNMP request tcp':'Exploitation',
'SNMP request udp':'Exploitation',
'TFTP Get':'Exploitation',
'WEB-CGI printenv access':'Exploitation',
'WEB-CGI redirect access':'Exploitation',
'WEB-FRONTPAGE /_vti_bin/ access':'Exploitation',
'WEB-FRONTPAGE _vti_rpc access':'Exploitation',
'WEB-FRONTPAGE shtml.dll access':'Exploitation',
'WEB-MISC /doc/ access':'Exploitation',
'WEB-MISC Invalid HTTP Version String':'Delivery',
'WEB-MISC PCT Client_Hello overflow attempt':'Exploitation',
'WEB-MISC SSLv2 Client_Hello with pad Challenge Length overflow attempt':'Delivery',
'WEB-MISC SSLv3 invalid data version attempt':'Delivery',
'WEB-MISC WEB-INF access':'Exploitation',
'WEB-MISC cross site scripting attempt':'Delivery',
'WEB-MISC http directory traversal':'Exploitation',
'WEB-MISC mod-plsql administration access':'Exploitation',
'WEB-MISC oracle portal demo access':'Exploitation',
'WEB-MISC robots.txt access':'Exploitation',
'WEB-PHP /_admin access':'Exploitation',
'WEB-PHP admin.php access':'Exploitation',
'WEB-PHP directory.php access':'Exploitation',
'WEB-PHP remote include path':'Exploitation',
'WEB-PHP viewtopic.php access':'Exploitation',
'X11 xopen':'Delivery',
'MALWARE-BACKDOOR - Dagger_1.4.0':'Exploitation',
'MALWARE-BACKDOOR QAZ Worm Client Login access':'Exploitation',
'MALWARE-BACKDOOR netbus getinfo':'Exploitation',
'MALWARE-BACKDOOR NetBus Pro 2.0 connection established':'Exploitation',
'MALWARE-BACKDOOR Infector.1.x':'Exploitation',
'MALWARE-BACKDOOR SatansBackdoor.2.0.Beta':'Exploitation',
'MALWARE-BACKDOOR Doly 2.0 access':'Exploitation',
'MALWARE-BACKDOOR Infector 1.6 Client to Server Connection Request':'Exploitation',
'MALWARE-BACKDOOR HackAttack 1.20 Connect':'Exploitation',
'PROTOCOL-FTP ADMw0rm ftp login attempt':'Delivery',
'MALWARE-BACKDOOR NetSphere access':'Exploitation',
'MALWARE-BACKDOOR GateCrasher':'Exploitation',
'MALWARE-BACKDOOR BackConstruction 2.1 Connection':'Exploitation',
'MALWARE-BACKDOOR BackConstruction 2.1 Client FTP Open Request':'Exploitation',
'MALWARE-BACKDOOR BackConstruction 2.1 Server FTP Open Reply':'Exploitation',
'MALWARE-BACKDOOR Matrix 2.0 Client connect':'Exploitation',
'MALWARE-BACKDOOR Matrix 2.0 Server access':'Exploitation',
'MALWARE-BACKDOOR WinCrash 1.0 Server Active':'Exploitation',
'MALWARE-BACKDOOR CDK':'Exploitation',
'MALWARE-BACKDOOR DeepThroat 3.1 Server Response':'Exploitation',
'MALWARE-BACKDOOR PhaseZero Server Active on Network':'Exploitation',
'MALWARE-BACKDOOR w00w00 attempt':'Exploitation',
'MALWARE-BACKDOOR attempt':'Exploitation',
'MALWARE-BACKDOOR MISC r00t attempt':'Exploitation',
'MALWARE-BACKDOOR MISC rewt attempt':'Exploitation',
'MALWARE-BACKDOOR MISC Linux rootkit attempt':'Exploitation',
'MALWARE-BACKDOOR MISC Linux rootkit attempt lrkr0x':'Exploitation',
'MALWARE-BACKDOOR MISC Linux rootkit attempt':'Exploitation',
'MALWARE-BACKDOOR MISC Linux rootkit satori attempt':'Exploitation',
'MALWARE-BACKDOOR MISC sm4ck attempt':'Exploitation',
'MALWARE-BACKDOOR MISC Solaris 2.5 attempt':'Exploitation',
'MALWARE-BACKDOOR HidePak backdoor attempt':'Exploitation',
'MALWARE-BACKDOOR HideSource backdoor attempt':'Exploitation',
'PROTOCOL-ICMP TFN Probe':'Reconaissance',
'PROTOCOL-ICMP tfn2k icmp possible communication':'Reconaissance',
'MALWARE-OTHER Trin00 Daemon to Master PONG message detected':'Exploitation',
'PROTOCOL-ICMP Stacheldraht server spoof':'Reconaissance',
'PROTOCOL-ICMP Stacheldraht gag server response':'Reconaissance',
'PROTOCOL-ICMP Stacheldraht server response':'Reconaissance',
'PROTOCOL-ICMP Stacheldraht client spoofworks':'Reconaissance',
'PROTOCOL-ICMP TFN client command BE':'Reconaissance',
'PROTOCOL-ICMP Stacheldraht client check skillz':'Reconaissance',
'MALWARE-OTHER shaft client login to handler':'Exploitation',
'MALWARE-OTHER Trin00 Daemon to Master message detected':'Exploitation',
'MALWARE-OTHER Trin00 Daemon to Master *HELLO* message detected':'Exploitation',
'MALWARE-OTHER Trin00 Attacker to Master default startup password':'Exploitation',
'MALWARE-OTHER Trin00 Attacker to Master default password':'Exploitation',
'MALWARE-OTHER Trin00 Attacker to Master default mdie password':'Exploitation',
'PROTOCOL-ICMP Stacheldraht client check gag':'Reconaissance',
'MALWARE-OTHER Trin00 Master to Daemon default password attempt':'Exploitation',
'PROTOCOL-ICMP TFN server response':'Reconaissance',
'MALWARE-OTHER shaft handler to agent':'Exploitation',
'MALWARE-OTHER shaft agent to handler':'Exploitation',
'MALWARE-OTHER mstream agent to handler':'Exploitation',
'MALWARE-OTHER mstream handler to agent':'Exploitation',
'MALWARE-OTHER mstream handler ping to agent':'Exploitation',
'MALWARE-OTHER mstream agent pong to handler':'Exploitation',
'MALWARE-OTHER mstream client to handler':'Exploitation',
'MALWARE-OTHER mstream handler to client':'Exploitation',
'MALWARE-OTHER mstream handler to client':'Exploitation',
'PROTOCOL-ICMP - TFN client command LE':'Reconaissance',
'PROTOCOL-DNS SPOOF query response PTR with TTL of 1 min. and no authority':'Reconaissance',
'PROTOCOL-DNS SPOOF query response with TTL of 1 min. and no authority':'Reconaissance',
'PROTOCOL-DNS dns zone transfer via TCP detected':'Information (NonCKC)',
'PROTOCOL-DNS named authors attempt':'Delivery',
'PROTOCOL-DNS named version attempt':'Delivery',
'SERVER-OTHER Bind Buffer Overflow via NXT records':'Exploitation',
'SERVER-OTHER Bind Buffer Overflow via NXT records named overflow ADM':'Exploitation',
'SERVER-OTHER Bind Buffer Overflow via NXT records named overflow ADMROCKS':'Exploitation',
'SERVER-OTHER Bind named overflow attempt':'Delivery',
'OS-LINUX OS-LINUX x86 Linux overflow attempt':'Delivery',
'OS-LINUX OS-LINUX x86 Linux overflow attempt':'Delivery',
'OS-LINUX OS-LINUX x86 Linux overflow attempt ADMv2':'Delivery',
'OS-OTHER OS-OTHER x86 FreeBSD overflow attempt':'Delivery',
'OS-SOLARIS EXPLOIT sparc overflow attempt':'Delivery',
'SERVER-OTHER UDP echo+chargen bomb':'Information (NonCKC)',
'OS-WINDOWS Microsoft WIndows IGMP dos attack':'Information (NonCKC)',
'PROTOCOL-ICMP ath':'Reconaissance',
'SERVER-OTHER RealNetworks Audio Server denial of service attempt':'Delivery',
'SERVER-OTHER RealNetworks Server template.html':'Information (NonCKC)',
'SERVER-OTHER RealNetworks Server template.html':'Information (NonCKC)',
'SERVER-OTHER Bay/Nortel Nautica Marlin':'Information (NonCKC)',
'SERVER-OTHER Ascend Route':'Information (NonCKC)',
'BROWSER-OTHER Netscape 4.7 client overflow':'Exploitation',
'PROTOCOL-POP EXPLOIT x86 BSD overflow':'Exploitation',
'PROTOCOL-POP EXPLOIT x86 BSD overflow':'Exploitation',
'PROTOCOL-POP EXPLOIT x86 Linux overflow':'Exploitation',
'PROTOCOL-POP EXPLOIT x86 SCO overflow':'Exploitation',
'PROTOCOL-POP EXPLOIT qpopper overflow':'Exploitation',
'OS-LINUX x86 Linux samba overflow':'Exploitation',
'OS-SOLARIS Oracle Solaris npls x86 overflow':'Exploitation',
'SERVER-OTHER LPRng overflow':'Exploitation',
'OS-LINUX Redhat 7.0 lprd overflow':'Exploitation',
'SERVER-OTHER Bind Buffer Overflow named tsig overflow attempt':'Delivery',
'SERVER-OTHER SCO calserver overflow':'Exploitation',
'SERVER-OTHER delegate proxy overflow':'Exploitation',
'SERVER-OTHER VQServer admin':'Information (NonCKC)',
'SERVER-OTHER CHAT IRC topic overflow':'Exploitation',
'SERVER-OTHER NextFTP client overflow':'Exploitation',
'SERVER-MAIL sniffit overflow':'Exploitation',
'SERVER-MAIL x86 windows MailMax overflow':'Exploitation',
'BROWSER-OTHER Netscape 4.7 unsucessful overflow':'Exploitation',
'OS-LINUX ntalkd x86 Linux overflow':'Exploitation',
'SERVER-OTHER Bind Buffer Overflow named tsig overflow attempt':'Delivery',
'OS-LINUX x86 Linux mountd overflow':'Exploitation',
'OS-LINUX x86 Linux mountd overflow':'Exploitation',
'OS-LINUX x86 Linux mountd overflow':'Exploitation',
'PROTOCOL-FINGER cmd_rootsh backdoor attempt':'Delivery',
'PROTOCOL-FINGER account enumeration attempt':'Delivery',
'PROTOCOL-FINGER search query':'Reconaissance',
'PROTOCOL-FINGER root query':'Reconaissance',
'PROTOCOL-FINGER null request':'Information (NonCKC)',
'PROTOCOL-FINGER remote command execution attempt':'Delivery',
'PROTOCOL-FINGER remote command pipe execution attempt':'Delivery',
'PROTOCOL-FINGER bomb attempt':'Delivery',
'PROTOCOL-FINGER redirection attempt':'Delivery',
'PROTOCOL-FINGER cybercop query':'Reconaissance',
'PROTOCOL-FINGER 0 query':'Reconaissance',
'PROTOCOL-FINGER . query':'Reconaissance',
'PROTOCOL-FTP .forward':'Information (NonCKC)',
'PROTOCOL-FTP .rhosts':'Information (NonCKC)',
'PROTOCOL-FTP CWD ~root attempt':'Delivery',
'PROTOCOL-FTP CEL overflow attempt':'Delivery',
'PROTOCOL-FTP adm scan':'Reconaissance',
'PROTOCOL-FTP iss scan':'Reconaissance',
'PROTOCOL-FTP pass wh00t':'Information (NonCKC)',
'PROTOCOL-FTP passwd retrieval attempt':'Delivery',
'PROTOCOL-FTP piss scan':'Reconaissance',
'PROTOCOL-FTP saint scan':'Reconaissance',
'PROTOCOL-FTP satan scan':'Reconaissance',
'PROTOCOL-FTP serv-u directory traversal':'Exploitation',
'PROTOCOL-FTP SITE EXEC attempt':'Delivery',
'PROTOCOL-FTP tar parameters':'Information (NonCKC)',
'PROTOCOL-ICMP IRDP router advertisement':'Reconaissance',
'PROTOCOL-ICMP IRDP router selection':'Reconaissance',
'PROTOCOL-ICMP PING undefined code':'Reconaissance',
'PROTOCOL-ICMP PING Unix':'Reconaissance',
'PROTOCOL-ICMP PING BSDtype':'Reconaissance',
'PROTOCOL-ICMP PING BayRS Router':'Reconaissance',
'PROTOCOL-ICMP PING BeOS4.x':'Reconaissance',
'PROTOCOL-ICMP PING Cisco Type.x':'Reconaissance',
'PROTOCOL-ICMP PING Delphi-Piette Windows':'Reconaissance',
'PROTOCOL-ICMP PING Flowpoint2200 or Network Management Software':'Reconaissance',
'PROTOCOL-ICMP PING IP NetMonitor Macintosh':'Reconaissance',
'PROTOCOL-ICMP PING LINUX/*BSD':'Reconaissance',
'PROTOCOL-ICMP PING Microsoft Windows':'Reconaissance',
'PROTOCOL-ICMP PING Network Toolbox 3 Windows':'Reconaissance',
'PROTOCOL-ICMP PING Ping-O-MeterWindows':'Reconaissance',
'PROTOCOL-ICMP PING Pinger Windows':'Reconaissance',
'PROTOCOL-ICMP PING Seer Windows':'Reconaissance',
'PROTOCOL-ICMP PING Oracle Solaris':'Reconaissance',
'PROTOCOL-ICMP PING Windows':'Reconaissance',
'PROTOCOL-ICMP PING':'Reconaissance',
'PROTOCOL-ICMP traceroute':'Reconaissance',
'PROTOCOL-ICMP Address Mask Reply':'Reconaissance',
'PROTOCOL-ICMP Address Mask Reply undefined code':'Reconaissance',
'PROTOCOL-ICMP Address Mask Request':'Reconaissance',
'PROTOCOL-ICMP Address Mask Request undefined code':'Reconaissance',
'PROTOCOL-ICMP Alternate Host Address':'Reconaissance',
'PROTOCOL-ICMP Alternate Host Address undefined code':'Reconaissance',
'PROTOCOL-ICMP Datagram Conversion Error':'Reconaissance',
'PROTOCOL-ICMP Datagram Conversion Error undefined code':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Destination Host Unknown':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Destination Network Unknown':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Fragmentation Needed and DF bit was set':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Host Precedence Violation':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Host Unreachable for Type of Service':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Host Unreachable':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Network Unreachable for Type of Service':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Network Unreachable':'Reconaissance',
'PROTOCOL-ICMP destination unreachable port unreachable packet detected':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Precedence Cutoff in effect':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Protocol Unreachable':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Source Host Isolated':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable Source Route Failed':'Reconaissance',
'PROTOCOL-ICMP Destination Unreachable cndefined code':'Reconaissance',
'PROTOCOL-ICMP Echo Reply':'Reconaissance',
'PROTOCOL-ICMP Echo Reply undefined code':'Reconaissance',
'PROTOCOL-ICMP Fragment Reassembly Time Exceeded':'Reconaissance',
'PROTOCOL-ICMP IPV6 I-Am-Here':'Reconaissance',
'PROTOCOL-ICMP IPV6 I-Am-Here undefined code':'Reconaissance',
'PROTOCOL-ICMP IPV6 Where-Are-You':'Reconaissance',
'PROTOCOL-ICMP IPV6 Where-Are-You undefined code':'Reconaissance',
'PROTOCOL-ICMP Information Reply':'Reconaissance',
'PROTOCOL-ICMP Information Reply undefined code':'Reconaissance',
'PROTOCOL-ICMP Information Request':'Reconaissance',
'PROTOCOL-ICMP Information Request undefined code':'Reconaissance',
'PROTOCOL-ICMP Mobile Host Redirect':'Reconaissance',
'PROTOCOL-ICMP Mobile Host Redirect undefined code':'Reconaissance',
'PROTOCOL-ICMP Mobile Registration Reply':'Reconaissance',
'PROTOCOL-ICMP Mobile Registration Reply undefined code':'Reconaissance',
'PROTOCOL-ICMP Mobile Registration Request':'Reconaissance',
'PROTOCOL-ICMP Mobile Registration Request undefined code':'Reconaissance',
'PROTOCOL-ICMP Parameter Problem Bad Length':'Reconaissance',
'PROTOCOL-ICMP Parameter Problem Missing a Required Option':'Reconaissance',
'PROTOCOL-ICMP Parameter Problem Unspecified Error':'Reconaissance',
'PROTOCOL-ICMP Parameter Problem undefined Code':'Reconaissance',
'PROTOCOL-ICMP Photuris Reserved':'Reconaissance',
'PROTOCOL-ICMP Photuris Unknown Security Parameters Index':'Reconaissance',
'PROTOCOL-ICMP Photuris Valid Security Parameters, But Authentication Failed':'Reconaissance',
'PROTOCOL-ICMP Photuris Valid Security Parameters, But Decryption Failed':'Reconaissance',
'PROTOCOL-ICMP Photuris undefined code!':'Reconaissance',
'PROTOCOL-ICMP Redirect for TOS and Host':'Reconaissance',
'PROTOCOL-ICMP Redirect for TOS and Network':'Reconaissance',
'PROTOCOL-ICMP Redirect undefined code':'Reconaissance',
'PROTOCOL-ICMP Reserved for Security Type 19':'Reconaissance',
'PROTOCOL-ICMP Reserved for Security Type 19 undefined code':'Reconaissance',
'PROTOCOL-ICMP Router Advertisement':'Reconaissance',
'PROTOCOL-ICMP Router Selection':'Reconaissance',
'PROTOCOL-ICMP SKIP':'Reconaissance',
'PROTOCOL-ICMP SKIP undefined code':'Reconaissance',
'PROTOCOL-ICMP Source Quench undefined code':'Reconaissance',
'PROTOCOL-ICMP Time-To-Live Exceeded in Transit':'Reconaissance',
'PROTOCOL-ICMP Time-To-Live Exceeded in Transit undefined code':'Reconaissance',
'PROTOCOL-ICMP Timestamp Reply':'Reconaissance',
'PROTOCOL-ICMP Timestamp Reply undefined code':'Reconaissance',
'PROTOCOL-ICMP Timestamp Request':'Reconaissance',
'PROTOCOL-ICMP Timestamp Request undefined code':'Reconaissance',
'PROTOCOL-ICMP Traceroute':'Reconaissance',
'PROTOCOL-ICMP Traceroute undefined code':'Reconaissance',
'PROTOCOL-ICMP unassigned type 1':'Reconaissance',
'PROTOCOL-ICMP unassigned type 1 undefined code':'Reconaissance',
'PROTOCOL-ICMP unassigned type 2':'Reconaissance',
'PROTOCOL-ICMP unassigned type 2 undefined code':'Reconaissance',
'PROTOCOL-ICMP unassigned type 7':'Reconaissance',
'PROTOCOL-ICMP unassigned type 7 undefined code':'Reconaissance',
'PROTOCOL-ICMP ISS Pinger':'Reconaissance',
'PROTOCOL-ICMP L3retriever Ping':'Reconaissance',
'PROTOCOL-ICMP Nemesis v1.1 Echo':'Reconaissance',
'PROTOCOL-ICMP superscan echo':'Reconaissance',
'PROTOCOL-ICMP webtrends scanner':'Reconaissance',
'PROTOCOL-ICMP PING speedera':'Reconaissance',
'PROTOCOL-ICMP TJPingPro1.1Build 2 Windows':'Reconaissance',
'PROTOCOL-ICMP PING WhatsupGold Windows':'Reconaissance',
'PROTOCOL-ICMP PING CyberKit 2.2 Windows':'Reconaissance',
'PROTOCOL-ICMP PING Sniffer Pro/NetXRay network scan':'Reconaissance',
'PROTOCOL-FTP no password':'Information (NonCKC)',
'SERVER-MAIL battle-mail traffic':'Information (NonCKC)',
'PROTOCOL-FTP Bad login':'Information (NonCKC)',
'PROTOCOL-TELNET login failed':'Information (NonCKC)',
'APP-DETECT psyBNC access':'Information (NonCKC)',
'INDICATOR-COMPROMISE command completed':'Information (NonCKC)',
'INDICATOR-COMPROMISE command error':'Information (NonCKC)',
'INDICATOR-COMPROMISE file copied ok':'Information (NonCKC)',
'INDICATOR-COMPROMISE id check returned root':'Information (NonCKC)',
'SERVER-OTHER Insecure TIMBUKTU Password':'Information (NonCKC)',
'PUA-OTHER PCAnywhere Attempted Administrator Login':'Delivery',
'SERVER-OTHER gopher proxy':'Information (NonCKC)',
'SERVER-WEBAPP PCCS mysql database admin tool access':'Information (NonCKC)',
'POLICY-OTHER HP JetDirect LCD modification attempt':'Delivery',
'PUA-OTHER PCAnywhere Failed Login':'Information (NonCKC)',
'SERVER-OTHER ramen worm':'Information (NonCKC)',
'PROTOCOL-SNMP NT UserList':'Information (NonCKC)',
'X11 xdmcp query':'Reconaissance',
'PROTOCOL-TFTP Put':'Information (NonCKC)',
'PROTOCOL-TFTP parent directory':'Information (NonCKC)',
'PROTOCOL-TFTP root directory':'Information (NonCKC)',
'NETBIOS DCERPC NCACN-IP-TCP srvsvc NetrShareEnum null policy handle attempt':'Delivery',
'OS-WINDOWS NT NULL session':'Information (NonCKC)',
'NETBIOS SMB CD..':'Information (NonCKC)',
'NETBIOS SMB CD...':'Information (NonCKC)',
'POLICY-SOCIAL Microsoft MSN message':'Information (NonCKC)',
'POLICY-SOCIAL ICQ access':'Information (NonCKC)',
'POLICY-SOCIAL IRC nick change':'Information (NonCKC)',
'INDICATOR-COMPROMISE FTP STOR 1MB possible warez site':'Information (NonCKC)',
'INDICATOR-COMPROMISE FTP RETR 1MB possible warez site':'Information (NonCKC)',
'INDICATOR-COMPROMISE FTP CWD /  possible warez site':'Information (NonCKC)',
'INDICATOR-COMPROMISE FTP CWD   possible warez site':'Information (NonCKC)',
'INDICATOR-COMPROMISE FTP MKD   possible warez site':'Information (NonCKC)',
'INDICATOR-COMPROMISE FTP MKD . possible warez site':'Information (NonCKC)',
'POLICY-OTHER FTP anonymous login attempt':'Delivery',
'INDICATOR-COMPROMISE FTP MKD /  possible warez site':'Information (NonCKC)',
'POLICY-OTHER WinGate telnet server response':'Information (NonCKC)',
'PUA-P2P Outbound GNUTella client request':'Information (NonCKC)',
'PUA-P2P GNUTella client request':'Information (NonCKC)',
'APP-DETECT VNC server response':'Information (NonCKC)',
'APP-DETECT PCAnywhere server response':'Information (NonCKC)',
'SERVER-MAIL SMTP relaying denied':'Information (NonCKC)',
'POLICY-OTHER HP JetDirect LCD modification attempt':'Delivery',
'PROTOCOL-RPC snmpXdmi overflow attempt TCP':'Delivery',
'PROTOCOL-RPC DOS ttdbserv Solaris':'Information (NonCKC)',
'PROTOCOL-RPC mountd TCP export request':'Information (NonCKC)',
'PROTOCOL-RPC portmap admind request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap amountd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap bootparam request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap cmsd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap mountd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap nisd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap pcnfsd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rexd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rstatd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rusers request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap sadmind request UDP attempt':'Delivery',
'PROTOCOL-RPC portmap selection_svc request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap status request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap ttdbserv request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap yppasswd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap ypserv request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap ypupdated request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap snmpXdmi request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap espd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap listing TCP 111':'Information (NonCKC)',
'PROTOCOL-RPC portmap listing TCP 32771':'Information (NonCKC)',
'PROTOCOL-SERVICES rlogin LinuxNIS':'Information (NonCKC)',
'PROTOCOL-SERVICES rlogin bin':'Information (NonCKC)',
'PROTOCOL-SERVICES rlogin echo++':'Information (NonCKC)',
'PROTOCOL-SERVICES Unix rlogin froot parameter root access attempt':'Delivery',
'PROTOCOL-SERVICES rlogin login failure':'Information (NonCKC)',
'PROTOCOL-SERVICES rlogin root':'Information (NonCKC)',
'PROTOCOL-SERVICES rsh bin':'Information (NonCKC)',
'PROTOCOL-SERVICES rsh echo + +':'Information (NonCKC)',
'PROTOCOL-SERVICES rsh froot':'Information (NonCKC)',
'PROTOCOL-SERVICES rsh root':'Information (NonCKC)',
'PROTOCOL-SERVICES rlogin login failure':'Information (NonCKC)',
'PROTOCOL-RPC rusers query UDP':'Reconaissance',
'INDICATOR-SCAN myscan':'Reconaissance',
'MALWARE-BACKDOOR hack-a-tack attempt':'Exploitation',
'INDICATOR-SCAN ident version request':'Reconaissance',
'INDICATOR-SCAN cybercop os probe':'Reconaissance',
'INDICATOR-SCAN ipEye SYN scan':'Reconaissance',
'INDICATOR-SCAN cybercop os PA12 attempt':'Reconaissance',
'INDICATOR-SCAN cybercop os SFU12 probe':'Reconaissance',
'INDICATOR-SCAN synscan portscan':'Reconaissance',
'SERVER-MAIL ehlo cybercop attempt':'Delivery',
'SERVER-MAIL expn cybercop attempt':'Delivery',
'INDICATOR-SCAN Amanda client-version request':'Reconaissance',
'INDICATOR-SCAN XTACACS logout':'Reconaissance',
'INDICATOR-SCAN cybercop udp bomb':'Reconaissance',
'INDICATOR-SCAN Webtrends Scanner UDP Probe':'Reconaissance',
'INDICATOR-SHELLCODE SGI NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE SGI NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE AIX NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE Digital UNIX NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE HP-UX NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE HP-UX NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE sparc NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE sparc NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE sparc NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE Oracle sparc setuid 0':'Information (NonCKC)',
'INDICATOR-SHELLCODE x86 NOOP':'Information (NonCKC)',
'INDICATOR-SHELLCODE x86 setgid 0':'Information (NonCKC)',
'INDICATOR-SHELLCODE x86 setuid 0':'Information (NonCKC)',
'INDICATOR-SHELLCODE Linux shellcode':'Information (NonCKC)',
'SERVER-MAIL RCPT TO overflow':'Exploitation',
'SERVER-MAIL Sendmail 8.6.9 exploit':'Information (NonCKC)',
'SERVER-MAIL Netmanager chameleon SMTPd buffer overflow attempt':'Delivery',
'SERVER-MAIL Microsoft Windows Exchange Server 5.5 mime DOS':'Information (NonCKC)',
'SERVER-MAIL Sendmail expn decode':'Information (NonCKC)',
'SERVER-MAIL expn root':'Information (NonCKC)',
'SERVER-MAIL Majordomo ifs':'Information (NonCKC)',
'SERVER-MAIL Sendmail 5.5.5 exploit':'Information (NonCKC)',
'SERVER-MAIL Sendmail rcpt to command attempt':'Delivery',
'SERVER-MAIL Sendmail RCPT TO decode attempt':'Delivery',
'SERVER-MAIL Sendmail 5.6.5 exploit':'Information (NonCKC)',
'SERVER-MAIL Sendmail 8.6.10 exploit':'Information (NonCKC)',
'SERVER-MAIL Sendmail 8.6.10 exploit':'Information (NonCKC)',
'SERVER-MAIL Sendmail 8.6.9 exploit':'Information (NonCKC)',
'SERVER-MAIL Sendmail 8.6.9 exploit':'Information (NonCKC)',
'SERVER-MAIL Sendmail 8.6.9c exploit':'Information (NonCKC)',
'SERVER-MAIL vrfy decode':'Information (NonCKC)',
'SQL sp_start_job - program execution':'Information (NonCKC)',
'SQL sp_start_job - program execution':'Information (NonCKC)',
'SQL sp_password password change':'Information (NonCKC)',
'SQL sp_delete_alert log file deletion':'Information (NonCKC)',
'SQL sp_adduser database user creation':'Information (NonCKC)',
'SQL xp_cmdshell program execution':'Information (NonCKC)',
'SQL sp_password - password change':'Information (NonCKC)',
'SQL sp_delete_alert log file deletion':'Information (NonCKC)',
'SQL sp_adduser - database user creation':'Information (NonCKC)',
'SERVER-MSSQL xp_reg* - registry access':'Information (NonCKC)',
'SQL xp_cmdshell - program execution':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'SERVER-MSSQL xp_reg* registry access':'Information (NonCKC)',
'INDICATOR-SHELLCODE shellcode attempt':'Delivery',
'INDICATOR-SHELLCODE shellcode attempt':'Delivery',
'INDICATOR-SHELLCODE shellcode attempt':'Delivery',
'INDICATOR-SHELLCODE shellcode attempt':'Delivery',
'SERVER-MSSQL xp_sprintf possible buffer overflow':'Exploitation',
'SERVER-MSSQL xp_sprintf possible buffer overflow':'Exploitation',
'PROTOCOL-TELNET 4Dgifts SGI account attempt':'Delivery',
'PROTOCOL-TELNET EZsetup account attempt':'Delivery',
'PROTOCOL-TELNET SGI telnetd format bug':'Information (NonCKC)',
'PROTOCOL-TELNET ld_library_path':'Information (NonCKC)',
'PROTOCOL-TELNET livingston DOS':'Information (NonCKC)',
'PROTOCOL-TELNET resolv_host_conf':'Information (NonCKC)',
'PROTOCOL-TELNET Attempted SU from wrong group':'Delivery',
'PROTOCOL-TELNET not on console':'Information (NonCKC)',
'PROTOCOL-TELNET login incorrect':'Information (NonCKC)',
'PROTOCOL-TELNET root login':'Information (NonCKC)',
'SERVER-WEBAPP HyperSeek hsx.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP SWSoft ASPSeek Overflow attempt':'Delivery',
'SERVER-WEBAPP Progress webspeed access':'Information (NonCKC)',
'SERVER-WEBAPP yabb directory traversal attempt':'Delivery',
'SERVER-WEBAPP /wwwboard/passwd.txt access':'Information (NonCKC)',
'SERVER-WEBAPP webdriver access':'Information (NonCKC)',
'SERVER-WEBAPP whois_raw.cgi arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP whois_raw.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP websitepro path access':'Information (NonCKC)',
'SERVER-WEBAPP webplus version access':'Information (NonCKC)',
'SERVER-WEBAPP webplus directory traversal':'Exploitation',
'SERVER-WEBAPP websendmail access':'Information (NonCKC)',
'SERVER-WEBAPP dcboard.cgi invalid user addition attempt':'Delivery',
'SERVER-WEBAPP dcforum.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP mmstdod.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP anaconda directory traversal attempt':'Delivery',
'SERVER-WEBAPP imagemap.exe overflow attempt':'Delivery',
'SERVER-WEBAPP cvsweb.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP php.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP glimpse access':'Information (NonCKC)',
'SERVER-WEBAPP htmlscript access':'Information (NonCKC)',
'SERVER-WEBAPP info2www access':'Information (NonCKC)',
'SERVER-WEBAPP maillist.pl access':'Information (NonCKC)',
'SERVER-WEBAPP nph-test-cgi access':'Information (NonCKC)',
'SERVER-WEBAPP perl.exe access':'Information (NonCKC)',
'SERVER-WEBAPP rguest.exe access':'Information (NonCKC)',
'SERVER-WEBAPP rwwwshell.pl access':'Information (NonCKC)',
'SERVER-WEBAPP test-cgi access':'Information (NonCKC)',
'SERVER-WEBAPP textcounter.pl access':'Information (NonCKC)',
'SERVER-WEBAPP uploader.exe access':'Information (NonCKC)',
'SERVER-WEBAPP webgais access':'Information (NonCKC)',
'SERVER-WEBAPP finger access':'Information (NonCKC)',
'SERVER-WEBAPP perlshop.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP aglimpse access':'Information (NonCKC)',
'SERVER-WEBAPP anform2 access':'Information (NonCKC)',
'SERVER-WEBAPP args.bat access':'Information (NonCKC)',
'SERVER-WEBAPP AT-admin.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP bnbform.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP campas access':'Information (NonCKC)',
'SERVER-WEBAPP view-source directory traversal':'Exploitation',
'SERVER-WEBAPP view-source access':'Information (NonCKC)',
'SERVER-WEBAPP wais.pl access':'Information (NonCKC)',
'SERVER-WEBAPP files.pl access':'Information (NonCKC)',
'SERVER-WEBAPP wguest.exe access':'Information (NonCKC)',
'SERVER-WEBAPP wrap access':'Information (NonCKC)',
'SERVER-WEBAPP classifieds.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP environ.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP faxsurvey access':'Information (NonCKC)',
'SERVER-WEBAPP filemail access':'Information (NonCKC)',
'SERVER-WEBAPP man.sh access':'Information (NonCKC)',
'SERVER-WEBAPP snork.bat access':'Information (NonCKC)',
'SERVER-WEBAPP w3-msql access':'Information (NonCKC)',
'SERVER-WEBAPP csh access':'Information (NonCKC)',
'SERVER-WEBAPP day5datacopier.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP day5datanotifier.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP ksh access':'Information (NonCKC)',
'SERVER-WEBAPP post-query access':'Reconaissance',
'SERVER-WEBAPP visadmin.exe access':'Information (NonCKC)',
'SERVER-WEBAPP rsh access':'Information (NonCKC)',
'SERVER-WEBAPP dumpenv.pl access':'Information (NonCKC)',
'SERVER-WEBAPP snorkerz.cmd access':'Information (NonCKC)',
'SERVER-WEBAPP survey.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP tcsh access':'Information (NonCKC)',
'SERVER-WEBAPP win-c-sample.exe access':'Information (NonCKC)',
'SERVER-WEBAPP rksh access':'Information (NonCKC)',
'SERVER-WEBAPP w3tvars.pm access':'Information (NonCKC)',
'SERVER-WEBAPP admin.pl access':'Information (NonCKC)',
'SERVER-WEBAPP LWGate access':'Information (NonCKC)',
'SERVER-WEBAPP archie access':'Information (NonCKC)',
'SERVER-WEBAPP calendar access':'Information (NonCKC)',
'SERVER-WEBAPP flexform access':'Information (NonCKC)',
'SERVER-WEBAPP bash access':'Information (NonCKC)',
'SERVER-WEBAPP phf access':'Information (NonCKC)',
'SERVER-WEBAPP www-sql access':'Information (NonCKC)',
'SERVER-WEBAPP wwwadmin.pl access':'Information (NonCKC)',
'SERVER-WEBAPP ppdscgi.exe access':'Information (NonCKC)',
'SERVER-WEBAPP sendform.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP upload.pl access':'Information (NonCKC)',
'SERVER-WEBAPP AnyForm2 access':'Information (NonCKC)',
'SERVER-WEBAPP bb-hist.sh access':'Information (NonCKC)',
'SERVER-WEBAPP redirect access':'Information (NonCKC)',
'SERVER-WEBAPP way-board access':'Information (NonCKC)',
'SERVER-WEBAPP pals-cgi access':'Information (NonCKC)',
'SERVER-WEBAPP commerce.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Amaya templates sendtemp.pl directory traversal attempt':'Delivery',
'SERVER-WEBAPP webspirs.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP webspirs.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP tstisapi.dll access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion cfcache.map access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion exampleapp application.cfm':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion application.cfm access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion getfile.cfm access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion addcontent.cfm access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion administrator access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion datasource username attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion fileexists.cfm access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion exprcalc access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion parks access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion cfappman access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion beaninfo access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion evaluate.cfm access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion getodbcdsn access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion db connections flush attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion expeval access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion datasource passwordattempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion datasource attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion admin encrypt attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion displayfile access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion getodbcin attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion admin decrypt attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion mainframeset access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion set odbc ini attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion settings refresh attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion exampleapp access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion CFUSION_VERIFYMAIL access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion snippets attempt':'Delivery',
'SERVER-OTHER Adobe Coldfusion cfmlsyntaxcheck.cfm access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion application.cfm access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion onrequestend.cfm access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion startstop DOS access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion gettempdirectory.cfm access ':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage _vti_rpc access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage posting':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage shtml.dll access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage contents.htm access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage orders.htm access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage fpsrvadm.exe access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage fpremadm.exe access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage fpadmin.htm access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage fpadmcgi.exe access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage orders.txt access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage form_results access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage registrations.htm access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage cfgwiz.exe access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage authors.pwd access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage author.exe access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage administrators.pwd access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage form_results.htm access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage access.cnf access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage register.txt access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage registrations.txt access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage service.cnf access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage service.pwd':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage service.stp access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage services.cnf access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage shtml.exe access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage svcacl.cnf access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage users.pwd access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage writeto.cnf access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage .... request':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage dvwssr.dll access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage register.htm access':'Information (NonCKC)',
'SERVER-IIS WebDAV file lock attempt':'Delivery',
'SERVER-IIS ISAPI .printer access':'Information (NonCKC)',
'SERVER-IIS *.idc attempt':'Delivery',
'SERVER-IIS Microsoft Windows IIS directory traversal attempt':'Delivery',
'SERVER-IIS Alternate Data streams ASP file access attempt':'Delivery',
'SERVER-WEBAPP .bat? access':'Information (NonCKC)',
'SERVER-IIS .cnf access':'Information (NonCKC)',
'SERVER-IIS ASP contents view':'Information (NonCKC)',
'SERVER-IIS ASP contents view':'Information (NonCKC)',
'SERVER-IIS CGImail.exe access':'Information (NonCKC)',
'SERVER-IIS JET VBA access':'Information (NonCKC)',
'SERVER-IIS JET VBA access':'Information (NonCKC)',
'SERVER-IIS MSProxy access':'Information (NonCKC)',
'FILE-IDENTIFY .htr access file download request':'Information (NonCKC)',
'MALWARE-CNC sensepost.exe command shell':'Exploitation',
'SERVER-OTHER Microsoft Frontpage _vti_inf.html access':'Information (NonCKC)',
'SERVER-IIS achg.htr access':'Information (NonCKC)',
'SERVER-IIS adctest.asp access':'Information (NonCKC)',
'SERVER-IIS iisadmin access':'Information (NonCKC)',
'SERVER-IIS /scripts/iisadmin/default.htm access':'Information (NonCKC)',
'SERVER-IIS ism.dll access':'Information (NonCKC)',
'SERVER-IIS anot.htr access':'Information (NonCKC)',
'SERVER-IIS asp-dot attempt':'Delivery',
'SERVER-IIS asp-srch attempt':'Delivery',
'SERVER-IIS bdir access':'Information (NonCKC)',
'SERVER-IIS bdir.htr access':'Information (NonCKC)',
'SERVER-WEBAPP carbo.dll access':'Information (NonCKC)',
'SERVER-IIS cmd.exe access':'Information (NonCKC)',
'SERVER-IIS cmd? access':'Information (NonCKC)',
'SERVER-IIS codebrowser Exair access':'Information (NonCKC)',
'SERVER-IIS codebrowser SDK access':'Information (NonCKC)',
'SERVER-IIS Form_JScript.asp access':'Information (NonCKC)',
'SERVER-IIS del attempt':'Delivery',
'SERVER-IIS directory listing':'Information (NonCKC)',
'SERVER-IIS encoding access':'Information (NonCKC)',
'SERVER-IIS exec-src access':'Information (NonCKC)',
'SERVER-IIS fpcount attempt':'Delivery',
'SERVER-IIS fpcount access':'Information (NonCKC)',
'SERVER-IIS getdrvs.exe access':'Information (NonCKC)',
'SERVER-IIS global.asa access':'Information (NonCKC)',
'SERVER-IIS idc-srch attempt':'Delivery',
'SERVER-IIS iisadmpwd attempt':'Delivery',
'SERVER-IIS Malformed Hit-Highlighting Argument File Access Attempt':'Delivery',
'SERVER-IIS isc$data attempt':'Delivery',
'SERVER-IIS ism.dll attempt':'Delivery',
'SERVER-IIS jet vba access':'Information (NonCKC)',
'SERVER-IIS msadcs.dll access':'Information (NonCKC)',
'SERVER-IIS newdsn.exe access':'Information (NonCKC)',
'SERVER-IIS perl access':'Information (NonCKC)',
'SERVER-IIS perl-browse newline attempt':'Delivery',
'SERVER-IIS perl-browse space attempt':'Delivery',
'SERVER-IIS query.asp access':'Reconaissance',
'SERVER-IIS scripts-browse access':'Information (NonCKC)',
'SERVER-IIS search97.vts access':'Information (NonCKC)',
'SERVER-IIS /SiteServer/Publishing/viewcode.asp access':'Information (NonCKC)',
'SERVER-IIS showcode access':'Information (NonCKC)',
'SERVER-IIS viewcode access':'Information (NonCKC)',
'SERVER-IIS viewcode access':'Information (NonCKC)',
'SERVER-IIS viewcode access':'Information (NonCKC)',
'SERVER-IIS viewcode access':'Information (NonCKC)',
'SERVER-IIS showcode.asp access':'Information (NonCKC)',
'SERVER-IIS site server config access':'Information (NonCKC)',
'SERVER-IIS srch.htm access':'Information (NonCKC)',
'SERVER-IIS srchadm access':'Information (NonCKC)',
'SERVER-IIS uploadn.asp access':'Information (NonCKC)',
'SERVER-IIS view source via translate header':'Information (NonCKC)',
'SERVER-IIS viewcode.asp access':'Information (NonCKC)',
'SERVER-IIS webhits access':'Information (NonCKC)',
'SERVER-IIS Unauthorized IP Access Attempt':'Delivery',
'SERVER-IIS site/iisamples access':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Enterprise DOS':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Enterprise directory listing attempt':'Delivery',
'SERVER-WEBAPP iPlanet GETPROPERTIES attempt':'Delivery',
'FILE-OTHER technote main.cgi file directory traversal attempt':'Delivery',
'SERVER-WEBAPP technote print.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP ads.cgi command execution attempt':'Delivery',
'SERVER-WEBAPP weblogic/tomcat .jsp view source attempt':'Delivery',
'SERVER-APACHE Apache Tomcat view source attempt':'Delivery',
'SQL ftp attempt':'Delivery',
'SQL xp_enumdsn attempt':'Delivery',
'SQL xp_filelist attempt':'Delivery',
'SQL xp_availablemedia attempt':'Delivery',
'SQL xp_cmdshell attempt':'Delivery',
'SERVER-WEBAPP nc.exe attempt':'Delivery',
'SERVER-WEBAPP wsh attempt':'Delivery',
'SERVER-WEBAPP rcmd attempt':'Delivery',
'SERVER-WEBAPP telnet attempt':'Delivery',
'SERVER-WEBAPP net attempt':'Delivery',
'SERVER-WEBAPP tftp attempt':'Delivery',
'SQL xp_regread attempt':'Delivery',
'SERVER-WEBAPP WebDAV search access':'Information (NonCKC)',
'SERVER-WEBAPP .htpasswd access':'Information (NonCKC)',
'SERVER-WEBAPP Lotus Domino directory traversal':'Exploitation',
'SERVER-WEBAPP webhits.exe access':'Information (NonCKC)',
'SERVER-IIS postinfo.asp access':'Information (NonCKC)',
'SERVER-IIS repost.asp access':'Information (NonCKC)',
'SQL queryhit.htm access':'Reconaissance',
'SQL counter.exe access':'Information (NonCKC)',
'OS-WINDOWS Microsoft Windows WebDAV propfind access':'Information (NonCKC)',
'SERVER-WEBAPP unify eWave ServletExec upload':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Servers suite DOS':'Information (NonCKC)',
'SERVER-WEBAPP amazon 1-click cookie theft':'Information (NonCKC)',
'SERVER-WEBAPP unify eWave ServletExec DOS':'Information (NonCKC)',
'SERVER-WEBAPP Allaire JRUN DOS attempt':'Delivery',
'SERVER-WEBAPP strings overflow':'Exploitation',
'SERVER-WEBAPP strings overflow':'Exploitation',
'SERVER-WEBAPP eXtropia webstore directory traversal':'Exploitation',
'SERVER-WEBAPP shopping cart directory traversal':'Exploitation',
'SERVER-WEBAPP Allaire Pro Web Shell attempt':'Delivery',
'SERVER-WEBAPP ICQ Webfront HTTP DOS':'Information (NonCKC)',
'SERVER-WEBAPP Armada Style Master Index directory traversal':'Exploitation',
'SERVER-WEBAPP cached_feed.cgi moreover shopping cart directory traversal':'Exploitation',
'SERVER-WEBAPP Talentsoft Web+ Source Code view access':'Information (NonCKC)',
'SERVER-WEBAPP Talentsoft Web+ internal IP Address access':'Information (NonCKC)',
'SERVER-WEBAPP Talentsoft Web+ exploit attempt':'Delivery',
'SERVER-WEBAPP SmartWin CyberOffice Shopping Cart access':'Information (NonCKC)',
'SERVER-WEBAPP cybercop scan':'Reconaissance',
'INDICATOR-SCAN L3retriever HTTP Probe':'Reconaissance',
'INDICATOR-SCAN Webtrends HTTP probe':'Reconaissance',
'SERVER-WEBAPP nessus 1.X 404 probe':'Information (NonCKC)',
'SERVER-WEBAPP Netscape admin passwd':'Information (NonCKC)',
'SERVER-WEBAPP BigBrother access':'Information (NonCKC)',
'SERVER-WEBAPP Poll-it access':'Information (NonCKC)',
'SERVER-WEBAPP ftp.pl access':'Information (NonCKC)',
'SERVER-APACHE Apache Tomcat server snoop access':'Information (NonCKC)',
'SERVER-WEBAPP ROXEN directory list attempt':'Delivery',
'SERVER-WEBAPP apache source.asp file access':'Information (NonCKC)',
'SERVER-APACHE Apache Tomcat server exploit access':'Information (NonCKC)',
'SERVER-WEBAPP ICQ webserver DOS':'Information (NonCKC)',
'SERVER-WEBAPP Lotus DelDoc attempt':'Delivery',
'SERVER-WEBAPP Lotus EditDoc attempt':'Delivery',
'SERVER-WEBAPP ls 20-l':'Information (NonCKC)',
'SERVER-WEBAPP mlog.phtml access':'Information (NonCKC)',
'SERVER-WEBAPP mylog.phtml access':'Information (NonCKC)',
'SERVER-WEBAPP /etc/passwd file access attempt':'Delivery',
'SERVER-WEBAPP ?PageServices access':'Information (NonCKC)',
'SERVER-WEBAPP Ecommerce check.txt access':'Information (NonCKC)',
'SERVER-WEBAPP webcart access':'Information (NonCKC)',
'SERVER-WEBAPP AuthChangeUrl access':'Information (NonCKC)',
'SERVER-WEBAPP convert.bas access':'Information (NonCKC)',
'SERVER-WEBAPP cpshost.dll access':'Information (NonCKC)',
'SERVER-WEBAPP .htaccess access':'Information (NonCKC)',
'SERVER-WEBAPP .wwwacl access':'Information (NonCKC)',
'SERVER-WEBAPP .wwwacl access':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Unixware overflow':'Exploitation',
'INDICATOR-SCAN cybercop os probe':'Reconaissance',
'SERVER-WEBAPP Phorum admin access':'Information (NonCKC)',
'SERVER-WEBAPP cd..':'Information (NonCKC)',
'SERVER-WEBAPP Phorum authentication access':'Information (NonCKC)',
'SERVER-WEBAPP whisker HEAD/./':'Information (NonCKC)',
'SERVER-WEBAPP guestbook.pl access':'Information (NonCKC)',
'SERVER-WEBAPP handler access':'Information (NonCKC)',
'SERVER-WEBAPP /.... access':'Information (NonCKC)',
'SERVER-WEBAPP root access':'Information (NonCKC)',
'SERVER-WEBAPP Ecommerce import.txt access':'Information (NonCKC)',
'SERVER-WEBAPP cat_ access':'Information (NonCKC)',
'SERVER-WEBAPP Ecommerce import.txt access':'Information (NonCKC)',
'SERVER-WEBAPP count.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Domino catalog.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino domcfg.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino domlog.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino log.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino names.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Ecommerce checks.txt access':'Information (NonCKC)',
'SERVER-WEBAPP apache directory disclosure attempt':'Delivery',
'SERVER-WEBAPP Netscape PublishingXpert access':'Information (NonCKC)',
'SERVER-WEBAPP windmail.exe access':'Information (NonCKC)',
'SERVER-WEBAPP webplus access':'Information (NonCKC)',
'SERVER-WEBAPP Netscape dir index wp':'Information (NonCKC)',
'SERVER-WEBAPP piranha passwd.php3 access':'Information (NonCKC)',
'SERVER-WEBAPP cart 32 AdminPwd access':'Information (NonCKC)',
'SERVER-WEBAPP webdist.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP shopping cart access':'Information (NonCKC)',
'SERVER-WEBAPP Novell Groupwise gwweb.exe access':'Information (NonCKC)',
'SERVER-WEBAPP ws_ftp.ini access':'Information (NonCKC)',
'SERVER-WEBAPP rpm_query access':'Reconaissance',
'SERVER-WEBAPP mall log order access':'Information (NonCKC)',
'SERVER-WEBAPP bigconf.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP architext_query.pl access':'Reconaissance',
'SERVER-WEBAPP /cgi-bin/jj access':'Information (NonCKC)',
'SERVER-WEBAPP wwwboard.pl access':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP Phorum read access':'Information (NonCKC)',
'SERVER-WEBAPP Phorum violation access':'Information (NonCKC)',
'SERVER-WEBAPP get32.exe access':'Information (NonCKC)',
'SERVER-WEBAPP Annex Terminal DOS attempt':'Delivery',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP bizdbsearch attempt':'Delivery',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP SalesLogix Eviewer web command attempt':'Delivery',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP Trend Micro OfficeScan access':'Reconaissance',
'SERVER-WEBAPP oracle web arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP sojourn.cgi File attempt':'Delivery',
'SERVER-WEBAPP sojourn.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP SGI InfoSearch fname attempt':'Delivery',
'SERVER-WEBAPP Phorum code access':'Information (NonCKC)',
'SERVER-WEBAPP Netscape Enterprise Server directory view':'Information (NonCKC)',
'SERVER-WEBAPP Compaq Insight directory traversal':'Exploitation',
'INDICATOR-COMPROMISE Invalid URL':'Information (NonCKC)',
'INDICATOR-COMPROMISE 403 Forbidden':'Information (NonCKC)',
'SERVER-WEBAPP search.vts access':'Information (NonCKC)',
'SERVER-WEBAPP ax-admin.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP axs.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP cachemgr.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP htgrep access':'Information (NonCKC)',
'SERVER-WEBAPP responder.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP .nsconfig access':'Information (NonCKC)',
'SERVER-WEBAPP web-map.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Admin_files access':'Information (NonCKC)',
'SERVER-WEBAPP backup access':'Information (NonCKC)',
'SERVER-WEBAPP intranet access':'Information (NonCKC)',
'SERVER-WEBAPP ministats admin access':'Information (NonCKC)',
'SERVER-WEBAPP filemail access':'Information (NonCKC)',
'SERVER-WEBAPP plusmail access':'Information (NonCKC)',
'SERVER-WEBAPP adminlogin access':'Information (NonCKC)',
'SERVER-WEBAPP dfire.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP ultraboard access':'Information (NonCKC)',
'SERVER-WEBAPP Muscat Empower cgi access':'Information (NonCKC)',
'SERVER-WEBAPP pals-cgi arbitrary file access attempt':'Delivery',
'SERVER-WEBAPP ROADS search.pl attempt':'Delivery',
'X11 MIT Magic Cookie detected':'Information (NonCKC)',
'X11 xopen':'Information (NonCKC)',
'PROTOCOL-FTP CWD ...':'Information (NonCKC)',
'SERVER-WEBAPP VirusWall FtpSave access':'Information (NonCKC)',
'SERVER-WEBAPP VirusWall catinfo access':'Information (NonCKC)',
'SERVER-WEBAPP VirusWall catinfo access':'Information (NonCKC)',
'SERVER-WEBAPP VirusWall FtpSaveCSP access':'Information (NonCKC)',
'SERVER-WEBAPP VirusWall FtpSaveCVP access':'Information (NonCKC)',
'OS-WINDOWS RFParalyze Attempt':'Delivery',
'SERVER-OTHER MDBMS overflow':'Exploitation',
'SERVER-WEBAPP SWEditServlet directory traversal attempt':'Delivery',
'SERVER-IIS ISAPI .ida access':'Information (NonCKC)',
'SERVER-IIS ISAPI .ida attempt':'Delivery',
'SERVER-IIS ISAPI .idq attempt':'Delivery',
'SERVER-IIS ISAPI .idq access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage rad fp30reg.dll access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage rad fp4areg.dll access':'Information (NonCKC)',
'OS-OTHER Cisco IOS HTTP configuration attempt':'Delivery',
'PROTOCOL-TELNET bsd telnet exploit response':'Information (NonCKC)',
'PROTOCOL-TELNET bsd exploit client finishing':'Information (NonCKC)',
'SERVER-WEBAPP PHPLIB remote command attempt':'Delivery',
'SERVER-WEBAPP PHPLIB remote command attempt':'Delivery',
'SERVER-IIS CodeRed v2 root.exe access':'Information (NonCKC)',
'SERVER-OTHER Winnuke attack':'Information (NonCKC)',
'SERVER-WEBAPP SWEditServlet access':'Information (NonCKC)',
'SERVER-OTHER AIX pdnsd overflow':'Exploitation',
'PROTOCOL-RPC portmap admind request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap amountd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap bootparam request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap cmsd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap nisd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap pcnfsd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rexd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rstatd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rusers request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap sadmind request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap selection_svc request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap ttdbserv request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap yppasswd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap ypserv request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap ypupdated request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap snmpXdmi request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap listing UDP 111':'Information (NonCKC)',
'PROTOCOL-RPC portmap listing UDP 32771':'Information (NonCKC)',
'SERVER-IIS Microsoft Office Outlook web dos':'Information (NonCKC)',
'SERVER-OTHER readme.eml download attempt':'Delivery',
'SERVER-IIS msdac access':'Information (NonCKC)',
'SERVER-IIS _mem_bin access':'Information (NonCKC)',
'SERVER-OTHER Microsoft Frontpage /_vti_bin/ access':'Information (NonCKC)',
'PROTOCOL-TFTP GET Admin.dll':'Information (NonCKC)',
'FILE-OTHER readme.eml autoload attempt':'Delivery',
'SERVER-WEBAPP sml3com access':'Information (NonCKC)',
'INDICATOR-COMPROMISE directory listing':'Information (NonCKC)',
'INDICATOR-COMPROMISE nimda RICHED20.DLL':'Information (NonCKC)',
'SERVER-WEBAPP admin.php file upload attempt':'Delivery',
'SERVER-WEBAPP admin.php access':'Information (NonCKC)',
'SERVER-WEBAPP console.exe access':'Information (NonCKC)',
'SERVER-WEBAPP cs.exe access':'Information (NonCKC)',
'SERVER-WEBAPP txt2html.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP txt2html.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP store.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP sendmessage.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP zsh access':'Information (NonCKC)',
'SERVER-OTHER rwhoisd format string attempt':'Delivery',
'INDICATOR-SHELLCODE ssh CRC32 overflow /bin/sh':'Exploitation',
'INDICATOR-SHELLCODE ssh CRC32 overflow filler':'Exploitation',
'INDICATOR-SHELLCODE ssh CRC32 overflow NOOP':'Exploitation',
'INDICATOR-SHELLCODE ssh CRC32 overflow':'Exploitation',
'SERVER-WEBAPP .htgroup access':'Information (NonCKC)',
'SERVER-WEBAPP sadmind worm access':'Information (NonCKC)',
'SERVER-WEBAPP jrun directory browse attempt':'Delivery',
'PROTOCOL-FTP wu-ftp bad file completion attempt':'Delivery',
'PROTOCOL-FTP wu-ftp bad file completion attempt':'Delivery',
'PROTOCOL-FTP STAT overflow attempt':'Delivery',
'SERVER-IIS Form_VBScript.asp access':'Information (NonCKC)',
'SERVER-WEBAPP Trend Micro OfficeScan attempt':'Reconaissance',
'SERVER-OTHER CHAT IRC Ettercap parse overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows UPnP malformed advertisement':'Information (NonCKC)',
'SERVER-WEBAPP mod-plsql administration access':'Information (NonCKC)',
'SERVER-MSSQL raiserror possible buffer overflow':'Exploitation',
'SQL raiserror possible buffer overflow':'Exploitation',
'OS-WINDOWS Microsoft Windows UPnP Location overflow attempt':'Delivery',
'INDICATOR-SHELLCODE x86 inc ebx NOOP':'Information (NonCKC)',
'SERVER-WEBAPP lastlines.cgi access':'Information (NonCKC)',
'POLICY-SOCIAL AIM AddGame attempt':'Delivery',
'INDICATOR-SHELLCODE x86 inc ecx NOOP':'Information (NonCKC)',
'SERVER-WEBAPP zml.cgi attempt':'Delivery',
'SERVER-WEBAPP zml.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP wayboard attempt':'Delivery',
'SERVER-OTHER CDE dtspcd exploit attempt':'Delivery',
'SERVER-WEBAPP PHP-Nuke remote file include attempt':'Delivery',
'SERVER-IIS /scripts/samples/ access':'Information (NonCKC)',
'SERVER-IIS /msadc/samples/ access':'Information (NonCKC)',
'SERVER-IIS iissamples access':'Information (NonCKC)',
'SERVER-WEBAPP AHG search.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP agora.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP smssend.php access':'Information (NonCKC)',
'SERVER-OTHER MSDTC attempt':'Delivery',
'PROTOCOL-SNMP community string buffer overflow attempt':'Delivery',
'SERVER-WEBAPP dcboard.cgi access':'Information (NonCKC)',
'PROTOCOL-SNMP public access udp':'Information (NonCKC)',
'PROTOCOL-SNMP public access tcp':'Information (NonCKC)',
'PROTOCOL-SNMP private access udp':'Information (NonCKC)',
'PROTOCOL-SNMP private access tcp':'Information (NonCKC)',
'PROTOCOL-SNMP Broadcast request':'Information (NonCKC)',
'PROTOCOL-SNMP broadcast trap':'Information (NonCKC)',
'PROTOCOL-SNMP request udp':'Information (NonCKC)',
'PROTOCOL-SNMP request tcp':'Information (NonCKC)',
'PROTOCOL-SNMP trap udp':'Information (NonCKC)',
'PROTOCOL-SNMP trap tcp':'Information (NonCKC)',
'PROTOCOL-SNMP AgentX/tcp request':'Information (NonCKC)',
'PROTOCOL-SNMP community string buffer overflow attempt with evasion':'Delivery',
'SERVER-WEBAPP content-disposition memchr overflow':'Exploitation',
'SERVER-WEBAPP content-disposition file upload attempt':'Delivery',
'PROTOCOL-SNMP PROTOS test-suite-req-app attempt':'Delivery',
'PROTOCOL-SNMP PROTOS test-suite-trap-app attempt':'Delivery',
'POLICY-MULTIMEDIA audio galaxy keepalive':'Information (NonCKC)',
'PUA-P2P GNUTella client request':'Information (NonCKC)',
'SERVER-WEBAPP .history access':'Information (NonCKC)',
'SERVER-WEBAPP .bash_history access':'Information (NonCKC)',
'PROTOCOL-DNS named authors attempt':'Delivery',
'POLICY-MULTIMEDIA Apple Quicktime User Agent access':'Information (NonCKC)',
'FILE-IDENTIFY Microsoft Windows Media download detected':'Information (NonCKC)',
'POLICY-MULTIMEDIA Shoutcast playlist redirection':'Information (NonCKC)',
'POLICY-MULTIMEDIA Icecast playlist redirection':'Information (NonCKC)',
'PROTOCOL-TFTP GET nc.exe':'Information (NonCKC)',
'PROTOCOL-TFTP GET shadow':'Information (NonCKC)',
'PROTOCOL-TFTP GET passwd':'Information (NonCKC)',
'PROTOCOL-TFTP Get':'Information (NonCKC)',
'INDICATOR-COMPROMISE FTP file_id.diz access possible warez site':'Information (NonCKC)',
'SERVER-MAIL vrfy root':'Information (NonCKC)',
'POLICY-OTHER Microsoft Windows Terminal server RDP attempt':'Delivery',
'POLICY-OTHER Microsoft Windows Terminal server request attempt':'Delivery',
'SERVER-MAIL Vintra Mailserver expn *@':'Information (NonCKC)',
'SERVER-WEBAPP NPH-maillist access':'Information (NonCKC)',
'SERVER-WEBAPP args.cmd access':'Information (NonCKC)',
'SERVER-WEBAPP AT-generated.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP wwwwais access':'Information (NonCKC)',
'SERVER-WEBAPP calendar.pl access':'Information (NonCKC)',
'SERVER-WEBAPP calender_admin.pl access':'Information (NonCKC)',
'SERVER-WEBAPP user_update_admin.pl access':'Information (NonCKC)',
'SERVER-WEBAPP user_update_passwd.pl access':'Information (NonCKC)',
'SERVER-WEBAPP bb-histlog.sh access':'Information (NonCKC)',
'SERVER-WEBAPP bb-histsvc.sh access':'Information (NonCKC)',
'SERVER-WEBAPP bb-rep.sh access':'Information (NonCKC)',
'SERVER-WEBAPP bb-replog.sh access':'Information (NonCKC)',
'POLICY-SOCIAL IRC message':'Information (NonCKC)',
'INDICATOR-COMPROMISE oracle one hour install':'Information (NonCKC)',
'SERVER-WEBAPP auktion.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP cgiforum.pl access':'Information (NonCKC)',
'SERVER-WEBAPP directorypro.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Web Shopper shopper.cgi attempt':'Delivery',
'SERVER-WEBAPP Web Shopper shopper.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP listrec.pl access':'Information (NonCKC)',
'SERVER-WEBAPP mailnews.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP book.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP newsdesk.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP cal_make.pl access':'Information (NonCKC)',
'SERVER-WEBAPP mailit.pl access':'Information (NonCKC)',
'SERVER-WEBAPP sdbsearch.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Simple Web Counter URI Parameter Buffer Overflow attempt':'Delivery',
'SERVER-WEBAPP ttawebtop.cgi arbitrary file attempt':'Delivery',
'SERVER-WEBAPP ttawebtop.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP upload.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP view_source access':'Information (NonCKC)',
'SERVER-WEBAPP ustorekeeper.pl access':'Information (NonCKC)',
'SERVER-IIS mkilog.exe access':'Information (NonCKC)',
'SERVER-IIS ctss.idc access':'Information (NonCKC)',
'SERVER-IIS /iisadmpwd/aexp2.htr access':'Information (NonCKC)',
'SERVER-WEBAPP store.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP nobody access':'Information (NonCKC)',
'SERVER-WEBAPP Phorum /support/common.php attempt':'Delivery',
'SERVER-WEBAPP Phorum /support/common.php access':'Information (NonCKC)',
'SERVER-WEBAPP RBS ISP /newuser  directory traversal attempt':'Delivery',
'SERVER-WEBAPP RBS ISP /newuser access':'Information (NonCKC)',
'SERVER-WEBAPP SIX webboard generate.cgi attempt':'Delivery',
'SERVER-WEBAPP SIX webboard generate.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP spin_client.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP SiteScope Service access':'Information (NonCKC)',
'SERVER-WEBAPP ExAir access':'Information (NonCKC)',
'SERVER-WEBAPP a1stats a1disp3.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP a1stats a1disp3.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP admentor admin.asp access':'Information (NonCKC)',
'POLICY-OTHER AFS access':'Information (NonCKC)',
'SERVER-WEBAPP alchemy http server PRN arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP alchemy http server NUL arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP alibaba.pl arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP alibaba.pl access':'Information (NonCKC)',
'SERVER-WEBAPP AltaVista Intranet Search directory traversal attempt':'Delivery',
'SERVER-WEBAPP test.bat arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP test.bat access':'Information (NonCKC)',
'SERVER-WEBAPP input.bat arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP input.bat access':'Information (NonCKC)',
'SERVER-WEBAPP input2.bat arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP input2.bat access':'Information (NonCKC)',
'SERVER-WEBAPP envout.bat arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP envout.bat access':'Information (NonCKC)',
'SERVER-WEBAPP nstelemetry.adp access':'Information (NonCKC)',
'SERVER-WEBAPP apache ?M=D directory list attempt':'Delivery',
'SERVER-WEBAPP server-info access':'Information (NonCKC)',
'SERVER-WEBAPP server-status access':'Information (NonCKC)',
'SERVER-WEBAPP ans.pl attempt':'Delivery',
'SERVER-WEBAPP ans.pl access':'Information (NonCKC)',
'SERVER-WEBAPP Axis Storpoint CD attempt':'Delivery',
'SERVER-WEBAPP Axis Storpoint CD access':'Information (NonCKC)',
'SERVER-WEBAPP basilix sendmail.inc access':'Information (NonCKC)',
'SERVER-WEBAPP basilix mysql.class access':'Information (NonCKC)',
'SERVER-WEBAPP BBoard access':'Information (NonCKC)',
'PROTOCOL-FTP SITE overflow attempt':'Delivery',
'SERVER-WEBAPP bb-hist.sh attempt':'Delivery',
'SERVER-WEBAPP bb-hostscv.sh attempt':'Delivery',
'SERVER-WEBAPP bb-hostscv.sh access':'Information (NonCKC)',
'SERVER-WEBAPP agora.cgi attempt':'Delivery',
'SERVER-WEBAPP bizdbsearch access':'Information (NonCKC)',
'SERVER-WEBAPP calendar_admin.pl arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP calendar_admin.pl access':'Information (NonCKC)',
'PROTOCOL-NNTP AUTHINFO USER overflow attempt':'Delivery',
'SERVER-WEBAPP /cgi-bin/ls access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion ?Mode=debug attempt':'Delivery',
'PROTOCOL-FINGER version query':'Reconaissance',
'SERVER-WEBAPP cgimail access':'Information (NonCKC)',
'SERVER-WEBAPP cgiwrap access':'Information (NonCKC)',
'SERVER-WEBAPP Cisco Catalyst command execution attempt':'Delivery',
'SERVER-OTHER Cisco denial of service attempt':'Delivery',
'SERVER-WEBAPP Cisco HTTP double-percent DOS attempt':'Delivery',
'SERVER-WEBAPP csSearch.cgi arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP csSearch.cgi access':'Information (NonCKC)',
'SERVER-MAIL HELO overflow attempt':'Delivery',
'SERVER-MAIL ETRN overflow attempt':'Delivery',
'SERVER-WEBAPP /CVS/Entries access':'Information (NonCKC)',
'SERVER-WEBAPP cvsweb version access':'Information (NonCKC)',
'SERVER-WEBAPP dbman db.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP DCShop access':'Information (NonCKC)',
'SERVER-WEBAPP DCShop orders.txt access':'Information (NonCKC)',
'SERVER-WEBAPP DCShop auth_user_file.txt access':'Information (NonCKC)',
'SERVER-WEBAPP Delegate whois overflow attempt':'Delivery',
'SERVER-WEBAPP /doc/packages access':'Information (NonCKC)',
'SERVER-WEBAPP /doc/ access':'Information (NonCKC)',
'PROTOCOL-FTP SITE CHOWN overflow attempt':'Delivery',
'SERVER-WEBAPP login.htm attempt':'Delivery',
'SERVER-WEBAPP login.htm access':'Information (NonCKC)',
'SERVER-WEBAPP eshop.pl arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP eshop.pl access':'Information (NonCKC)',
'SERVER-IIS /exchange/root.asp attempt':'Delivery',
'SERVER-IIS /exchange/root.asp access':'Information (NonCKC)',
'SERVER-WEBAPP loadpage.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP loadpage.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP dcforum.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP commerce.cgi arbitrary file access attempt':'Delivery',
'SERVER-WEBAPP cgiforum.pl attempt':'Delivery',
'SERVER-WEBAPP directorypro.cgi attempt':'Delivery',
'SERVER-WEBAPP Domino mab.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino cersvr.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino setup.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino statrep.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino webadmin.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino events4.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino ntsync4.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino collect4.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino mailw46.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino bookmark.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino agentrunner.nsf access':'Information (NonCKC)',
'SERVER-WEBAPP Domino mail.box access':'Information (NonCKC)',
'SERVER-WEBAPP cgitest.exe access':'Information (NonCKC)',
'SERVER-WEBAPP SalesLogix Eviewer access':'Information (NonCKC)',
'SERVER-WEBAPP musicat empower attempt':'Delivery',
'SERVER-WEBAPP faqmanager.cgi arbitrary file access attempt':'Delivery',
'SERVER-WEBAPP faqmanager.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP /fcgi-bin/echo.exe access':'Information (NonCKC)',
'SERVER-WEBAPP FormHandler.cgi external site redirection attempt':'Delivery',
'SERVER-WEBAPP FormHandler.cgi access':'Information (NonCKC)',
'SERVER-IIS htimage.exe access':'Information (NonCKC)',
'SERVER-WEBAPP guestbook.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Home Free search.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP search.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP htsearch arbitrary configuration file attempt':'Delivery',
'SERVER-WEBAPP htsearch arbitrary file read attempt':'Delivery',
'SERVER-WEBAPP htsearch access':'Information (NonCKC)',
'SERVER-WEBAPP DELETE attempt':'Delivery',
'SERVER-WEBAPP iChat directory traversal attempt':'Delivery',
'SERVER-OTHER iParty DOS attempt':'Delivery',
'SERVER-WEBAPP icat access':'Information (NonCKC)',
'SERVER-WEBAPP HyperSeek hsx.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP htmlscript attempt':'Delivery',
'SERVER-WEBAPP formmail arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP eXtropia webstore access':'Information (NonCKC)',
'SERVER-WEBAPP ftp.pl attempt':'Delivery',
'SERVER-WEBAPP handler attempt':'Delivery',
'SERVER-WEBAPP Novell Groupwise gwweb.exe attempt':'Delivery',
'SERVER-WEBAPP htgrep attempt':'Delivery',
'PROTOCOL-DNS named version attempt':'Delivery',
'SERVER-WEBAPP Bugzilla doeditvotes.cgi access':'Information (NonCKC)',
'SERVER-IIS .asp chunked Transfer-Encoding':'Information (NonCKC)',
'PROTOCOL-FTP CMD overflow attempt':'Delivery',
'PROTOCOL-FTP RNFR ././ attempt':'Delivery',
'PROTOCOL-FTP invalid MODE':'Information (NonCKC)',
'PROTOCOL-FTP PWD overflow attempt':'Delivery',
'PROTOCOL-FTP SYST overflow attempt':'Delivery',
'SERVER-IIS /StoreCSVS/InstantOrder.asmx request':'Information (NonCKC)',
'SERVER-WEBAPP FormHandler.cgi directory traversal attempt attempt':'Delivery',
'POLICY-SOCIAL AIM login':'Information (NonCKC)',
'PROTOCOL-POP PASS overflow attempt':'Delivery',
'PROTOCOL-POP APOP overflow attempt':'Delivery',
'SERVER-OTHER Xtramail Username overflow attempt':'Delivery',
'SERVER-WEBAPP yabb access':'Information (NonCKC)',
'INDICATOR-SCAN SSH Version map attempt':'Reconaissance',
'POLICY-SOCIAL IRC DCC file transfer request':'Information (NonCKC)',
'POLICY-SOCIAL IRC DCC chat request':'Information (NonCKC)',
'SERVER-OTHER DB2 dos attempt':'Delivery',
'SERVER-WEBAPP document.d2w access':'Information (NonCKC)',
'SERVER-WEBAPP db2www access':'Information (NonCKC)',
'SERVER-WEBAPP test-cgi attempt':'Delivery',
'SERVER-WEBAPP testcgi access':'Information (NonCKC)',
'SERVER-WEBAPP test.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP perl.exe command attempt':'Delivery',
'SERVER-WEBAPP perl command attempt':'Delivery',
'SERVER-WEBAPP tst.bat access':'Information (NonCKC)',
'SERVER-WEBAPP environ.pl access':'Information (NonCKC)',
'SERVER-WEBAPP campas attempt':'Delivery',
'SERVER-WEBAPP cart32.exe access':'Information (NonCKC)',
'SERVER-WEBAPP pfdispaly.cgi arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP pfdispaly.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP pagelog.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP pagelog.cgi access':'Information (NonCKC)',
'SERVER-OTHER Adobe Coldfusion sendmail.cfm access':'Information (NonCKC)',
'SERVER-IIS trace.axd access':'Information (NonCKC)',
'SERVER-IIS cmd32.exe access':'Information (NonCKC)',
'SERVER-WEBAPP /~ftp access':'Information (NonCKC)',
'SERVER-WEBAPP *%20.pl access':'Information (NonCKC)',
'SERVER-WEBAPP mkplog.exe access':'Information (NonCKC)',
'INDICATOR-COMPROMISE index of /cgi-bin/ response':'Information (NonCKC)',
'SERVER-WEBAPP cross site scripting HTML Image tag set to javascript attempt':'Delivery',
'SERVER-WEBAPP /cgi-bin/ access':'Information (NonCKC)',
'SERVER-WEBAPP /cgi-dos/ access':'Information (NonCKC)',
'SERVER-WEBAPP /home/ftp access':'Information (NonCKC)',
'SERVER-WEBAPP /home/www access':'Information (NonCKC)',
'PROTOCOL-FTP CWD ~ attempt':'Delivery',
'SERVER-ORACLE EXECUTE_SYSTEM attempt':'Delivery',
'SERVER-ORACLE connect_data remote version detection attempt':'Delivery',
'SERVER-ORACLE misparsed login response':'Information (NonCKC)',
'SERVER-ORACLE select union attempt':'Delivery',
'SERVER-ORACLE select like % attempt':'Delivery',
'SERVER-ORACLE select like % attempt backslash escaped':'Delivery',
'SERVER-ORACLE describe attempt':'Delivery',
'SERVER-ORACLE all_constraints access':'Information (NonCKC)',
'SERVER-ORACLE all_views access':'Information (NonCKC)',
'SERVER-ORACLE all_source access':'Information (NonCKC)',
'SERVER-ORACLE all_tables access':'Information (NonCKC)',
'SERVER-ORACLE all_tab_columns access':'Information (NonCKC)',
'SERVER-ORACLE all_tab_privs access':'Information (NonCKC)',
'SERVER-ORACLE dba_tablespace access':'Information (NonCKC)',
'SERVER-ORACLE dba_tables access':'Information (NonCKC)',
'SERVER-ORACLE user_tablespace access':'Information (NonCKC)',
'SERVER-ORACLE sys.all_users access':'Information (NonCKC)',
'SERVER-ORACLE grant attempt':'Delivery',
'SERVER-ORACLE ALTER USER attempt':'Delivery',
'SERVER-ORACLE drop table attempt':'Delivery',
'SERVER-ORACLE create table attempt':'Delivery',
'SERVER-ORACLE alter table attempt':'Delivery',
'SERVER-ORACLE truncate table attempt':'Delivery',
'SERVER-ORACLE create database attempt':'Delivery',
'SERVER-ORACLE alter database attempt':'Delivery',
'SERVER-WEBAPP imagemap.exe access':'Information (NonCKC)',
'SERVER-WEBAPP calendar-admin.pl access':'Information (NonCKC)',
'SERVER-WEBAPP Amaya templates sendtemp.pl access':'Information (NonCKC)',
'SERVER-WEBAPP auktion.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP cal_make.pl directory traversal attempt':'Delivery',
'SERVER-WEBAPP echo.bat arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP echo.bat access':'Information (NonCKC)',
'SERVER-WEBAPP hello.bat arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP hello.bat access':'Information (NonCKC)',
'SERVER-WEBAPP ad.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP bbs_forum.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP bsguest.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP bslist.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP cgforum.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP newdesk access':'Information (NonCKC)',
'SERVER-WEBAPP register.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP gbook.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP simplestguest.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP statsconfig.pl access':'Information (NonCKC)',
'SERVER-WEBAPP talkback.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP talkback.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP adcycle access':'Information (NonCKC)',
'SERVER-WEBAPP MachineInfo access':'Information (NonCKC)',
'SERVER-WEBAPP emumail.cgi NULL attempt':'Delivery',
'SERVER-WEBAPP emumail.cgi access':'Information (NonCKC)',
'SERVER-IIS +.htr code fragment attempt':'Delivery',
'SERVER-IIS doctodep.btr access':'Information (NonCKC)',
'SERVER-WEBAPP SGI InfoSearch fname access':'Information (NonCKC)',
'POLICY-SOCIAL IRC channel join':'Information (NonCKC)',
'SERVER-WEBAPP ustorekeeper.pl directory traversal attempt':'Delivery',
'SERVER-WEBAPP a1stats access':'Information (NonCKC)',
'PROTOCOL-RPC portmap rwalld request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rwalld request TCP':'Information (NonCKC)',
'PROTOCOL-FTP USER overflow attempt':'Delivery',
'BROWSER-OTHER Mozilla Netscape XMLHttpRequest local file read attempt':'Delivery',
'SERVER-WEBAPP squirrel mail spell-check arbitrary command attempt':'Delivery',
'SERVER-WEBAPP squirrel mail theme arbitrary command attempt':'Delivery',
'SERVER-WEBAPP global.inc access':'Information (NonCKC)',
'SERVER-WEBAPP DNSTools administrator authentication bypass attempt':'Delivery',
'SERVER-WEBAPP DNSTools authentication bypass attempt':'Delivery',
'SERVER-WEBAPP DNSTools access':'Information (NonCKC)',
'SERVER-WEBAPP Blahz-DNS dostuff.php modify user attempt':'Delivery',
'SERVER-WEBAPP Blahz-DNS dostuff.php access':'Information (NonCKC)',
'SERVER-WEBAPP SecureSite authentication bypass attempt':'Delivery',
'SERVER-WEBAPP Messagerie supp_membre.php access':'Information (NonCKC)',
'PROTOCOL-RPC portmap cachefsd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap cachefsd request TCP':'Information (NonCKC)',
'SERVER-IIS users.xml access':'Information (NonCKC)',
'SERVER-OTHER cachefsd buffer overflow attempt':'Delivery',
'POLICY-SOCIAL AIM AddExternalApp attempt':'Delivery',
'SERVER-IIS as_web.exe access':'Information (NonCKC)',
'SERVER-IIS as_web4.exe access':'Information (NonCKC)',
'PROTOCOL-IMAP partial body buffer overflow attempt':'Delivery',
'SERVER-IIS NewsPro administration authentication attempt':'Delivery',
'SERVER-WEBAPP b2 arbitrary command execution attempt':'Delivery',
'SQL xp_cmdshell program execution 445':'Information (NonCKC)',
'SERVER-WEBAPP phf arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP Nortel Contivity cgiproc DOS attempt':'Delivery',
'SERVER-WEBAPP Nortel Contivity cgiproc DOS attempt':'Delivery',
'SERVER-WEBAPP Nortel Contivity cgiproc access':'Information (NonCKC)',
'SERVER-WEBAPP search.dll directory listing attempt':'Delivery',
'SERVER-WEBAPP search.dll access':'Information (NonCKC)',
'SERVER-WEBAPP .DS_Store access':'Information (NonCKC)',
'SERVER-WEBAPP .FBCIndex access':'Information (NonCKC)',
'POLICY-OTHER IPSec PGPNet connection attempt':'Delivery',
'SERVER-IIS pbserver access':'Information (NonCKC)',
'SERVER-WEBAPP php.exe access':'Information (NonCKC)',
'SERVER-WEBAPP bb_smilies.php access':'Information (NonCKC)',
'SERVER-MYSQL root login attempt':'Delivery',
'SERVER-MYSQL show databases attempt':'Delivery',
'PROTOCOL-FTP EXPLOIT STAT asterisk dos attempt':'Delivery',
'PROTOCOL-FTP EXPLOIT STAT ? dos attempt':'Delivery',
'SERVER-WEBAPP csPassword.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP csPassword password.cgi.tmp access':'Information (NonCKC)',
'POLICY-SOCIAL IRC dns request':'Information (NonCKC)',
'POLICY-SOCIAL IRC dns response':'Information (NonCKC)',
'PROTOCOL-NNTP return code buffer overflow attempt':'Delivery',
'SERVER-IIS .asa HTTP header buffer overflow attempt':'Delivery',
'SERVER-IIS .cer HTTP header buffer overflow attempt':'Delivery',
'SERVER-IIS .cdx HTTP header buffer overflow attempt':'Delivery',
'SERVER-WEBAPP Oracle Reports CGI access':'Information (NonCKC)',
'SERVER-IIS .htr chunked Transfer-Encoding':'Information (NonCKC)',
'POLICY-OTHER Chunked-Encoding transfer attempt':'Delivery',
'SERVER-WEBAPP apache chunked encoding memory corruption exploit attempt':'Delivery',
'SERVER-APACHE Apache Chunked-Encoding worm attempt':'Delivery',
'SERVER-OTHER successful gobbles ssh exploit GOBBLE':'Information (NonCKC)',
'SERVER-OTHER successful gobbles ssh exploit uname':'Information (NonCKC)',
'SERVER-OTHER gobbles SSH exploit attempt':'Delivery',
'PROTOCOL-ICMP digital island bandwidth query':'Reconaissance',
'SERVER-WEBAPP CISCO VoIP DOS ATTEMPT':'Delivery',
'SERVER-WEBAPP directory.php arbitrary command attempt':'Delivery',
'SERVER-WEBAPP directory.php access':'Information (NonCKC)',
'SERVER-IIS MS Site Server default login attempt':'Delivery',
'SERVER-IIS MS Site Server admin attempt':'Delivery',
'SERVER-OTHER Alcatel PABX 4400 connection attempt':'Delivery',
'SERVER-WEBAPP IBM Net.Commerce orderdspc.d2w access':'Information (NonCKC)',
'SERVER-OTHER LPD dvips remote command execution attempt':'Delivery',
'SERVER-WEBAPP AlienForm alienform.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP AlienForm af.cgi directory traversal attempt':'Delivery',
'SERVER-WEBAPP AlienForm alienform.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP AlienForm af.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP WEB-INF access':'Information (NonCKC)',
'SERVER-APACHE Apache Tomcat servlet mapping cross site scripting attempt':'Delivery',
'SERVER-WEBAPP iPlanet Search directory traversal attempt':'Delivery',
'SERVER-APACHE Apache Tomcat TroubleShooter servlet access':'Information (NonCKC)',
'SERVER-APACHE Apache Tomcat SnoopServlet servlet access':'Information (NonCKC)',
'SERVER-WEBAPP jigsaw dos attempt':'Delivery',
'POLICY-SOCIAL ICQ forced user addition':'Information (NonCKC)',
'SERVER-WEBAPP PHP-Wiki cross site scripting attempt':'Delivery',
'SERVER-WEBAPP Macromedia SiteSpring cross site scripting attempt':'Delivery',
'SERVER-OTHER SSH server banner overflow':'Exploitation',
'SERVER-WEBAPP mailman cross site scripting attempt':'Delivery',
'FILE-JAVA Oracle Javascript document.domain attempt':'Delivery',
'BROWSER-FIREFOX Mozilla 1.0 Javascript arbitrary cookie access attempt':'Delivery',
'PROTOCOL-IMAP login buffer overflow attempt':'Delivery',
'MALWARE-BACKDOOR trinity connection attempt':'Exploitation',
'PROTOCOL-IMAP authenticate overflow attempt':'Delivery',
'PROTOCOL-IMAP list literal overflow attempt':'Delivery',
'POLICY-MULTIMEDIA vncviewer Java applet download attempt':'Delivery',
'SERVER-WEBAPP webalizer access':'Information (NonCKC)',
'SERVER-WEBAPP webcart-lite access':'Information (NonCKC)',
'SERVER-WEBAPP webfind.exe access':'Information (NonCKC)',
'SERVER-WEBAPP way-board.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP active.log access':'Information (NonCKC)',
'SERVER-WEBAPP robots.txt access':'Information (NonCKC)',
'MALWARE-BACKDOOR win-trin00 connection attempt':'Exploitation',
'PROTOCOL-ICMP Stacheldraht handler->agent niggahbitch':'Reconaissance',
'PROTOCOL-ICMP Stacheldraht agent->handler skillz':'Reconaissance',
'PROTOCOL-ICMP Stacheldraht handler->agent ficken':'Reconaissance',
'SERVER-WEBAPP robot.txt access':'Information (NonCKC)',
'SERVER-WEBAPP CISCO PIX Firewall Manager directory traversal attempt':'Delivery',
'SERVER-WEBAPP Oracle JavaServer default password login attempt':'Delivery',
'SERVER-WEBAPP Linksys router default password login attempt':'Delivery',
'SERVER-WEBAPP Linksys router default username and password login attempt':'Delivery',
'SERVER-WEBAPP mrtg.cgi directory traversal attempt':'Delivery',
'PROTOCOL-FTP SITE NEWER attempt':'Delivery',
'SERVER-WEBAPP webdist.cgi arbitrary command attempt':'Delivery',
'PROTOCOL-POP USER overflow attempt':'Delivery',
'X11 xdmcp info query':'Reconaissance',
'SERVER-WEBAPP Interactive Story story.pl arbitrary file read attempt':'Delivery',
'SERVER-WEBAPP Interactive Story story.pl access':'Information (NonCKC)',
'SERVER-WEBAPP siteUserMod.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Oracle XSQLConfig.xml access':'Information (NonCKC)',
'SERVER-WEBAPP Oracle Dynamic Monitoring Services dms access':'Information (NonCKC)',
'SERVER-WEBAPP globals.jsa access':'Information (NonCKC)',
'SERVER-WEBAPP Oracle Java Process Manager access':'Information (NonCKC)',
'SERVER-WEBAPP cgicso access':'Information (NonCKC)',
'SERVER-WEBAPP nph-publish.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP printenv access':'Information (NonCKC)',
'SERVER-WEBAPP sdbsearch.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP book.cgi arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP oracle web application server access':'Information (NonCKC)',
'SERVER-WEBAPP bad HTTP 1.1 request - potential worm attack':'Information (NonCKC)',
'INDICATOR-COMPROMISE id check returned userid':'Information (NonCKC)',
'SERVER-OTHER OpenSSL Worm traffic':'Information (NonCKC)',
'PROTOCOL-FTP SITE CPWD overflow attempt':'Delivery',
'MALWARE-CNC slapper worm admin traffic':'Exploitation',
'PROTOCOL-RPC status GHBN format string attack':'Information (NonCKC)',
'PROTOCOL-RPC status GHBN format string attack':'Information (NonCKC)',
'PROTOCOL-SNMP null community string attempt':'Delivery',
'PROTOCOL-SNMP missing community string attempt':'Delivery',
'INDICATOR-SHELLCODE kadmind buffer overflow attempt':'Delivery',
'INDICATOR-SHELLCODE kadmind buffer overflow attempt':'Delivery',
'INDICATOR-SHELLCODE kadmind buffer overflow attempt':'Delivery',
'INDICATOR-SHELLCODE kadmind buffer overflow attempt':'Delivery',
'INDICATOR-SHELLCODE kadmind buffer overflow attempt':'Delivery',
'INDICATOR-SHELLCODE kadmind buffer overflow attempt':'Delivery',
'SERVER-OTHER successful kadmind buffer overflow attempt':'Delivery',
'SERVER-OTHER successful kadmind buffer overflow attempt':'Delivery',
'PROTOCOL-IMAP lsub literal overflow attempt':'Delivery',
'PROTOCOL-IMAP rename overflow attempt':'Delivery',
'PROTOCOL-IMAP find overflow attempt':'Delivery',
'PROTOCOL-RPC AMD UDP amqproc_mount plog overflow attempt':'Delivery',
'PROTOCOL-RPC AMD TCP amqproc_mount plog overflow attempt':'Delivery',
'PROTOCOL-RPC CMSD UDP CMSD_CREATE buffer overflow attempt':'Delivery',
'PROTOCOL-RPC CMSD TCP CMSD_CREATE buffer overflow attempt':'Delivery',
'PROTOCOL-RPC CMSD TCP CMSD_INSERT buffer overflow attempt':'Delivery',
'PROTOCOL-RPC CMSD udp CMSD_INSERT buffer overflow attempt':'Delivery',
'PROTOCOL-RPC sadmind UDP NETMGT_PROC_SERVICE CLIENT_DOMAIN overflow attempt':'Delivery',
'PROTOCOL-RPC sadmind TCP NETMGT_PROC_SERVICE CLIENT_DOMAIN overflow attempt':'Delivery',
'PROTOCOL-RPC STATD UDP stat mon_name format string exploit attempt':'Delivery',
'PROTOCOL-RPC STATD TCP stat mon_name format string exploit attempt':'Delivery',
'PROTOCOL-RPC STATD UDP monitor mon_name format string exploit attempt':'Delivery',
'PROTOCOL-RPC STATD TCP monitor mon_name format string exploit attempt':'Delivery',
'INDICATOR-SCAN UPnP service discover attempt':'Reconaissance',
'PROTOCOL-ICMP SolarWinds IP scan attempt':'Reconaissance',
'PROTOCOL-FTP CWD overflow attempt':'Delivery',
'PROTOCOL-FTP SITE NEWER overflow attempt':'Delivery',
'PROTOCOL-FTP SITE ZIPCHK overflow attempt':'Delivery',
'PROTOCOL-RPC portmap proxy attempt TCP':'Delivery',
'PROTOCOL-RPC portmap proxy attempt UDP':'Delivery',
'PROTOCOL-RPC mountd UDP export request':'Information (NonCKC)',
'PROTOCOL-RPC mountd TCP exportall request':'Information (NonCKC)',
'PROTOCOL-RPC mountd UDP exportall request':'Information (NonCKC)',
'PROTOCOL-FTP authorized_keys':'Information (NonCKC)',
'PROTOCOL-FTP shadow retrieval attempt':'Delivery',
'PROTOCOL-IMAP auth literal overflow attempt':'Delivery',
'SERVER-WEBAPP rpc-nlog.pl access':'Information (NonCKC)',
'SERVER-WEBAPP rpc-smb.pl access':'Information (NonCKC)',
'SERVER-WEBAPP cart.cgi access':'Information (NonCKC)',
'PROTOCOL-POP AUTH overflow attempt':'Delivery',
'PROTOCOL-POP LIST overflow attempt':'Delivery',
'PROTOCOL-POP XTND overflow attempt':'Delivery',
'SERVER-OTHER bootp hardware address length overflow':'Exploitation',
'SERVER-OTHER bootp invalid hardware type':'Information (NonCKC)',
'PROTOCOL-TFTP GET filename overflow attempt':'Delivery',
'PROTOCOL-FTP RMDIR overflow attempt':'Delivery',
'SERVER-WEBAPP /Carello/add.exe access':'Information (NonCKC)',
'SERVER-WEBAPP /ecscripts/ecware.exe access':'Information (NonCKC)',
'SERVER-WEBAPP answerbook2 admin attempt':'Delivery',
'SERVER-WEBAPP answerbook2 arbitrary command execution attempt':'Delivery',
'PROTOCOL-DNS dns zone transfer via UDP detected':'Information (NonCKC)',
'PROTOCOL-RPC portmap SET attempt TCP 111':'Delivery',
'PROTOCOL-RPC portmap SET attempt UDP 111':'Delivery',
'PROTOCOL-RPC mountd TCP mount request':'Information (NonCKC)',
'PROTOCOL-RPC mountd UDP mount request':'Information (NonCKC)',
'PROTOCOL-RPC AMD TCP pid request':'Information (NonCKC)',
'PROTOCOL-RPC AMD UDP pid request':'Information (NonCKC)',
'PROTOCOL-RPC AMD TCP version request':'Information (NonCKC)',
'PROTOCOL-RPC AMD UDP version request':'Information (NonCKC)',
'PROTOCOL-RPC sadmind UDP PING':'Information (NonCKC)',
'PROTOCOL-RPC sadmind TCP PING':'Information (NonCKC)',
'PROTOCOL-RPC portmap NFS request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap NFS request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap RQUOTA request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap RQUOTA request TCP':'Information (NonCKC)',
'PROTOCOL-RPC RQUOTA getquota overflow attempt UDP':'Delivery',
'PROTOCOL-RPC tooltalk UDP overflow attempt':'Delivery',
'PROTOCOL-RPC tooltalk TCP overflow attempt':'Delivery',
'SERVER-OTHER GlobalSunTech Access Point Information Disclosure attempt':'Delivery',
'SERVER-WEBAPP phpbb quick-reply.php arbitrary command attempt':'Delivery',
'SERVER-WEBAPP phpbb quick-reply.php access':'Information (NonCKC)',
'SERVER-WEBAPP ion-p access':'Information (NonCKC)',
'SERVER-IIS MDAC Content-Type overflow attempt':'Delivery',
'PROTOCOL-FTP SITE EXEC format string attempt':'Delivery',
'PROTOCOL-FTP PASS overflow attempt':'Delivery',
'PROTOCOL-FTP MKD overflow attempt':'Delivery',
'PROTOCOL-FTP REST overflow attempt':'Delivery',
'PROTOCOL-FTP DELE overflow attempt':'Delivery',
'PROTOCOL-FTP RMD overflow attempt':'Delivery',
'SERVER-WEBAPP xp_regwrite attempt':'Delivery',
'SERVER-WEBAPP xp_regdeletekey attempt':'Delivery',
'SERVER-WEBAPP perl post attempt':'Delivery',
'MALWARE-BACKDOOR DeepThroat 3.1 Connection':'Exploitation',
'MALWARE-BACKDOOR DeepThroat 3.1 Connection attempt on port 3150':'Exploitation',
'MALWARE-BACKDOOR DeepThroat 3.1 Server Response on port 3150':'Exploitation',
'MALWARE-BACKDOOR DeepThroat 3.1 Connection attempt on port 4120':'Exploitation',
'MALWARE-BACKDOOR DeepThroat 3.1 Server Response on port 4120':'Exploitation',
'MALWARE-BACKDOOR Doly variant outbound connection attempt':'Exploitation',
'POLICY-SOCIAL Microsoft MSN outbound file transfer request':'Information (NonCKC)',
'SERVER-OTHER xfs overflow attempt':'Delivery',
'POLICY-SOCIAL Microsoft MSN outbound file transfer accept':'Information (NonCKC)',
'POLICY-SOCIAL Microsoft MSN outbound file transfer rejected':'Information (NonCKC)',
'POLICY-SOCIAL Microsoft MSN user search':'Information (NonCKC)',
'POLICY-SOCIAL Microsoft MSN login attempt':'Delivery',
'PROTOCOL-FTP LIST directory traversal attempt':'Delivery',
'PROTOCOL-IMAP login literal buffer overflow attempt':'Delivery',
'SERVER-WEBAPP vpasswd.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP alya.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP viralator.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP read_body.php access attempt':'Delivery',
'SERVER-WEBAPP calendar.php access':'Information (NonCKC)',
'SERVER-WEBAPP edit_image.php access':'Information (NonCKC)',
'SERVER-WEBAPP readmsg.php access':'Information (NonCKC)',
'SERVER-WEBAPP smartsearch.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP remote include path attempt':'Delivery',
'SQL Worm propagation attempt':'Delivery',
'SQL Worm propagation attempt OUTBOUND':'Delivery',
'PROTOCOL-RPC portmap kcms_server request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap kcms_server request TCP':'Information (NonCKC)',
'PROTOCOL-RPC kcms_server directory traversal attempt':'Delivery',
'INDICATOR-COMPROMISE CVS invalid user authentication response':'Information (NonCKC)',
'INDICATOR-COMPROMISE CVS invalid repository response':'Information (NonCKC)',
'INDICATOR-COMPROMISE CVS double free exploit attempt response':'Delivery',
'INDICATOR-COMPROMISE CVS invalid directory response':'Information (NonCKC)',
'INDICATOR-COMPROMISE CVS missing cvsroot response':'Information (NonCKC)',
'INDICATOR-COMPROMISE CVS invalid module response':'Information (NonCKC)',
'PROTOCOL-RPC portmap UNSET attempt TCP 111':'Delivery',
'PROTOCOL-RPC portmap UNSET attempt UDP 111':'Delivery',
'PROTOCOL-RPC portmap status request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap espd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC mountd TCP dump request':'Information (NonCKC)',
'PROTOCOL-RPC mountd UDP dump request':'Information (NonCKC)',
'PROTOCOL-RPC mountd TCP unmount request':'Information (NonCKC)',
'PROTOCOL-RPC mountd UDP unmount request':'Information (NonCKC)',
'PROTOCOL-RPC mountd TCP unmountall request':'Information (NonCKC)',
'PROTOCOL-RPC mountd UDP unmountall request':'Information (NonCKC)',
'PROTOCOL-RPC RQUOTA getquota overflow attempt TCP':'Delivery',
'PROTOCOL-RPC yppasswd username overflow attempt UDP':'Delivery',
'PROTOCOL-RPC yppasswd username overflow attempt TCP':'Delivery',
'PROTOCOL-RPC yppasswd old password overflow attempt UDP':'Delivery',
'PROTOCOL-RPC yppasswd old password overflow attempt TCP':'Delivery',
'PROTOCOL-RPC yppasswd new password overflow attempt UDP':'Delivery',
'PROTOCOL-RPC yppasswd new password overflow attempt TCP':'Delivery',
'PROTOCOL-RPC yppasswd user update UDP':'Information (NonCKC)',
'PROTOCOL-RPC yppasswd user update TCP':'Information (NonCKC)',
'PROTOCOL-RPC ypserv maplist request UDP':'Information (NonCKC)',
'PROTOCOL-RPC ypserv maplist request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap network-status-monitor request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap network-status-monitor request TCP':'Information (NonCKC)',
'PROTOCOL-RPC network-status-monitor mon-callback request UDP':'Information (NonCKC)',
'PROTOCOL-RPC network-status-monitor mon-callback request TCP':'Information (NonCKC)',
'SERVER-OTHER bootp hostname format string attempt':'Delivery',
'POLICY-OTHER xtacacs login attempt':'Delivery',
'INDICATOR-SCAN xtacacs failed login response':'Reconaissance',
'POLICY-OTHER xtacacs accepted login response':'Information (NonCKC)',
'INDICATOR-SCAN isakmp login failed':'Reconaissance',
'POLICY-OTHER PPTP Start Control Request attempt':'Delivery',
'PROTOCOL-RPC snmpXdmi overflow attempt UDP':'Delivery',
'PROTOCOL-IMAP partial body.peek buffer overflow attempt':'Delivery',
'SERVER-OTHER rsyncd module list access':'Information (NonCKC)',
'SQL ping attempt':'Delivery',
'SERVER-MSSQL version overflow attempt':'Delivery',
'SERVER-WEBAPP cached_feed.cgi moreover shopping cart access':'Information (NonCKC)',
'SERVER-WEBAPP overflow.cgi access':'Exploitation',
'SERVER-WEBAPP Bugtraq process_bug.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Bugtraq enter_bug.cgi arbitrary command attempt':'Delivery',
'SERVER-WEBAPP Bugtraq enter_bug.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP TRACE attempt':'Delivery',
'SERVER-WEBAPP helpout.exe access':'Information (NonCKC)',
'SERVER-WEBAPP MsmMask.exe attempt':'Delivery',
'SERVER-WEBAPP MsmMask.exe access':'Information (NonCKC)',
'SERVER-WEBAPP DB4Web access':'Information (NonCKC)',
'SERVER-APACHE Apache Tomcat null byte directory listing attempt':'Delivery',
'SERVER-WEBAPP iPlanet .perf access':'Information (NonCKC)',
'SERVER-WEBAPP Demarc SQL injection attempt':'Delivery',
'SERVER-WEBAPP Lotus Notes .csp script source download attempt':'Delivery',
'SERVER-WEBAPP Lotus Notes .pl script source download attempt':'Delivery',
'SERVER-WEBAPP Lotus Notes .exe script source download attempt':'Delivery',
'SERVER-WEBAPP BitKeeper arbitrary command attempt':'Delivery',
'SERVER-WEBAPP chip.ini access':'Information (NonCKC)',
'SERVER-WEBAPP post32.exe arbitrary command attempt':'Delivery',
'SERVER-WEBAPP post32.exe access':'Information (NonCKC)',
'SERVER-WEBAPP lyris.pl access':'Information (NonCKC)',
'SERVER-WEBAPP globals.pl access':'Information (NonCKC)',
'SERVER-WEBAPP Mambo uploadimage.php upload php file attempt':'Delivery',
'SERVER-WEBAPP Mambo upload.php upload php file attempt':'Delivery',
'SERVER-WEBAPP Mambo uploadimage.php access':'Information (NonCKC)',
'SERVER-WEBAPP Mambo upload.php access':'Information (NonCKC)',
'SERVER-WEBAPP phpBB privmsg.php access':'Information (NonCKC)',
'PROTOCOL-RPC portmap nlockmgr request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap nlockmgr request TCP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rpc.xfsmd request UDP':'Information (NonCKC)',
'PROTOCOL-RPC portmap rpc.xfsmd request TCP':'Information (NonCKC)',
'PROTOCOL-RPC rpc.xfsmd xfs_export attempt UDP':'Delivery',
'PROTOCOL-RPC rpc.xfsmd xfs_export attempt TCP':'Delivery',
'SERVER-WEBAPP parse_xml.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP streaming server parse_xml.cgi access':'Information (NonCKC)',
'SERVER-MAIL From comment overflow attempt':'Delivery',
'PROTOCOL-RPC ypupdated arbitrary command attempt UDP':'Delivery',
'PROTOCOL-RPC ypupdated arbitrary command attempt TCP':'Delivery',
'SERVER-IIS WEBDAV exploit attempt':'Delivery',
'SERVER-IIS WEBDAV nessus safe scan attempt':'Reconaissance',
'PROTOCOL-RPC portmap proxy integer overflow attempt UDP':'Delivery',
'PROTOCOL-RPC portmap proxy integer overflow attempt TCP':'Delivery',
'PROTOCOL-RPC CMSD UDP CMSD_CREATE array buffer overflow attempt':'Delivery',
'PROTOCOL-RPC CMSD TCP CMSD_CREATE array buffer overflow attempt':'Delivery',
'MALWARE-BACKDOOR SubSeven 2.1 Gold server connection response':'Exploitation',
'OS-WINDOWS Microsoft Windows SMB Trans Max Param/Count OS-WINDOWS attempt':'Delivery',
'NETBIOS SMB Trans2 OPEN2 unicode maximum param count overflow attempt':'Delivery',
'INDICATOR-COMPROMISE rexec username too long response':'Information (NonCKC)',
'PROTOCOL-IMAP authenticate literal overflow attempt':'Delivery',
'PROTOCOL-IMAP lsub overflow attempt':'Delivery',
'PROTOCOL-IMAP create buffer overflow attempt':'Delivery',
'PROTOCOL-POP CAPA overflow attempt':'Delivery',
'PROTOCOL-POP TOP overflow attempt':'Delivery',
'PROTOCOL-POP STAT overflow attempt':'Delivery',
'PROTOCOL-POP DELE overflow attempt':'Delivery',
'PROTOCOL-POP RSET overflow attempt':'Delivery',
'PROTOCOL-SERVICES rexec username overflow attempt':'Delivery',
'PROTOCOL-SERVICES rexec password overflow attempt':'Delivery',
'SERVER-WEBAPP album.pl access':'Information (NonCKC)',
'SERVER-WEBAPP chipcfg.cgi access':'Information (NonCKC)',
'SERVER-IIS Battleaxe Forum login.asp access':'Information (NonCKC)',
'PROTOCOL-IMAP list overflow attempt':'Delivery',
'PROTOCOL-IMAP rename literal overflow attempt':'Delivery',
'PROTOCOL-IMAP create literal buffer overflow attempt':'Delivery',
'PROTOCOL-POP DELE negative argument attempt':'Delivery',
'PROTOCOL-POP UIDL negative argument attempt':'Delivery',
'INDICATOR-COMPROMISE Microsoft cmd.exe banner':'Information (NonCKC)',
'MALWARE-BACKDOOR Remote PC Access connection':'Exploitation',
'PROTOCOL-FTP CWD Root directory traversal attempt':'Delivery',
'OS-WINDOWS Microsoft Windows PPTP Start Control Request buffer overflow attempt':'Delivery',
'SERVER-WEBAPP ikonboard.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP swsrv.cgi access':'Information (NonCKC)',
'SERVER-IIS nsiislog.dll access':'Information (NonCKC)',
'SERVER-IIS IISProtect siteadmin.asp access':'Information (NonCKC)',
'SERVER-IIS IISProtect access':'Information (NonCKC)',
'SERVER-IIS Synchrologic Email Accelerator userid list access attempt':'Delivery',
'SERVER-IIS MS BizTalk server access':'Information (NonCKC)',
'SERVER-IIS register.asp access':'Information (NonCKC)',
'SERVER-WEBAPP philboard.mdb access':'Information (NonCKC)',
'SERVER-WEBAPP philboard_admin.asp authentication bypass attempt':'Delivery',
'SERVER-WEBAPP philboard_admin.asp access':'Information (NonCKC)',
'SERVER-WEBAPP logicworks.ini access':'Information (NonCKC)',
'SERVER-WEBAPP /*.shtml access':'Information (NonCKC)',
'SERVER-WEBAPP p-news.php access':'Information (NonCKC)',
'SERVER-WEBAPP shoutbox.php directory traversal attempt':'Delivery',
'SERVER-WEBAPP shoutbox.php access':'Information (NonCKC)',
'SERVER-WEBAPP b2 cafelog gm-2-b2.php remote file include attempt':'Delivery',
'SERVER-WEBAPP b2 cafelog gm-2-b2.php access':'Information (NonCKC)',
'SERVER-WEBAPP TextPortal admin.php default password admin attempt':'Delivery',
'SERVER-WEBAPP TextPortal admin.php default password 12345 attempt':'Delivery',
'SERVER-WEBAPP BLNews objects.inc.php4 remote file include attempt':'Delivery',
'SERVER-WEBAPP BLNews objects.inc.php4 access':'Information (NonCKC)',
'SERVER-WEBAPP Turba status.php access':'Information (NonCKC)',
'SERVER-WEBAPP ttCMS header.php remote file include attempt':'Delivery',
'SERVER-WEBAPP ttCMS header.php access':'Information (NonCKC)',
'SERVER-WEBAPP test.php access':'Information (NonCKC)',
'SERVER-WEBAPP autohtml.php directory traversal attempt':'Delivery',
'SERVER-WEBAPP autohtml.php access':'Information (NonCKC)',
'SERVER-WEBAPP ttforum remote file include attempt':'Delivery',
'SERVER-WEBAPP mod_gzip_status access':'Information (NonCKC)',
'SERVER-IIS IISProtect globaladmin.asp access':'Information (NonCKC)',
'SERVER-OTHER BGP invalid length':'Information (NonCKC)',
'SERVER-OTHER BGP invalid type 0':'Information (NonCKC)',
'OS-WINDOWS Microsoft Windows SMB startup folder access':'Information (NonCKC)',
'OS-WINDOWS Microsoft Windows SMB startup folder unicode access':'Information (NonCKC)',
'PROTOCOL-FTP USER format string attempt':'Delivery',
'PROTOCOL-FTP PASS format string attempt':'Delivery',
'PUA-P2P BitTorrent announce request':'Information (NonCKC)',
'PUA-P2P BitTorrent transfer':'Information (NonCKC)',
'SERVER-MAIL Sendmail Content-Transfer-Encoding overflow attempt':'Delivery',
'PROTOCOL-RPC mountd TCP mount path overflow attempt':'Delivery',
'NETBIOS DCERPC invalid bind attempt':'Delivery',
'NETBIOS SMB DCERPC invalid bind attempt':'Delivery',
'SERVER-WEBAPP CSMailto.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP alert.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP catgy.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP cvsview2.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP cvslog.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP multidiff.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP dnewsweb.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Matt Wright download.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Webmin Directory edit_action.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Leif M. Wright everythingform.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP EasyBoard 2000 ezadmin.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP EasyBoard 2000 ezboard.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP EasyBoard 2000 ezman.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP FileSeek fileseek.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Faq-O-Matic fom.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Infonautics getdoc.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Multiple Vendors global.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Lars Ellingsen guestserver.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP cgiCentral WebStore imageFolio.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Oatmeal Studios Mail File mailfile.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP 3R Soft MailStudio 2000 mailview.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Alabanza Control Panel nsManager.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Ipswitch IMail readmail.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Ipswitch IMail printmail.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Oracle Cobalt RaQ service.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Trend Micro Interscan VirusWall setpasswd.cgi access':'Reconaissance',
'SERVER-WEBAPP Leif M. Wright simplestmail.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP cgiCentral WebStore ws_mail.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Infinity CGI exploit scanner nph-exploitscanget.cgi access':'Reconaissance',
'SERVER-WEBAPP CGIScript.net csNews.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Psunami Bulletin Board psunami.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP Linksys BEFSR41 gozila.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP pmachine remote file include attempt':'Delivery',
'SERVER-WEBAPP forum_details.php access':'Information (NonCKC)',
'SERVER-WEBAPP phpMyAdmin db_details_importdocsql.php access':'Information (NonCKC)',
'SERVER-WEBAPP viewtopic.php access':'Information (NonCKC)',
'SERVER-WEBAPP NetGear router default password login attempt admin/password':'Delivery',
'SERVER-WEBAPP register.dll access':'Information (NonCKC)',
'SERVER-WEBAPP ContentFilter.dll access':'Information (NonCKC)',
'SERVER-WEBAPP SFNofitication.dll access':'Information (NonCKC)',
'SERVER-WEBAPP TOP10.dll access':'Information (NonCKC)',
'SERVER-WEBAPP SpamExcp.dll access':'Information (NonCKC)',
'SERVER-WEBAPP spamrule.dll access':'Information (NonCKC)',
'SERVER-WEBAPP cgiWebupdate.exe access':'Information (NonCKC)',
'SERVER-WEBAPP WebLogic ConsoleHelp view source attempt':'Delivery',
'SERVER-WEBAPP redirect.exe access':'Information (NonCKC)',
'SERVER-WEBAPP changepw.exe access':'Information (NonCKC)',
'SERVER-WEBAPP cwmail.exe access':'Information (NonCKC)',
'SERVER-WEBAPP ddicgi.exe access':'Information (NonCKC)',
'SERVER-WEBAPP ndcgi.exe access':'Information (NonCKC)',
'SERVER-WEBAPP VsSetCookie.exe access':'Information (NonCKC)',
'SERVER-WEBAPP Webnews.exe access':'Information (NonCKC)',
'SERVER-WEBAPP webadmin.dll access':'Information (NonCKC)',
'SERVER-IIS UploadScript11.asp access':'Information (NonCKC)',
'SERVER-IIS DirectoryListing.asp access':'Information (NonCKC)',
'SERVER-IIS /pcadmin/login.asp access':'Information (NonCKC)',
'PROTOCOL-POP USER format string attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB-DS DCERPC Remote Activation bind attempt':'Delivery',
'SERVER-MAIL XEXCH50 overflow attempt':'Delivery',
'PROTOCOL-RPC sadmind query with root credentials attempt TCP':'Delivery',
'PROTOCOL-RPC sadmind query with root credentials attempt UDP':'Delivery',
'OS-WINDOWS DCERPC Messenger Service buffer overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB-DS DCERPC Messenger Service buffer overflow attempt':'Delivery',
'SERVER-MAIL EXPN overflow attempt':'Delivery',
'SERVER-MAIL VRFY overflow attempt':'Delivery',
'SERVER-MAIL Sendmail SEND FROM prescan too many addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail SEND FROM prescan too long addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail SAML FROM prescan too many addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail SAML FROM prescan too long addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail SOML FROM prescan too many addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail SOML FROM prescan too long addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail MAIL FROM prescan too many addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail MAIL FROM prescan too long addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail RCPT TO prescan too many addresses overflow':'Reconaissance',
'SERVER-MAIL Sendmail RCPT TO prescan too long addresses overflow':'Reconaissance',
'MALWARE-BACKDOOR FsSniffer connection attempt':'Exploitation',
'PROTOCOL-FTP LIST integer overflow attempt':'Delivery',
'PROTOCOL-IMAP login brute force attempt':'Delivery',
'PROTOCOL-POP login brute force attempt':'Delivery',
'SERVER-MAIL AUTH LOGON brute force attempt':'Delivery',
'SERVER-WEBAPP oracle portal demo access':'Information (NonCKC)',
'SERVER-WEBAPP PeopleSoft PeopleBooks psdoccgi access':'Information (NonCKC)',
'SERVER-WEBAPP client negative Content-Length attempt':'Delivery',
'SERVER-WEBAPP UpdateClasses.php access':'Information (NonCKC)',
'SERVER-WEBAPP Title.php access':'Information (NonCKC)',
'SERVER-WEBAPP Setup.php access':'Information (NonCKC)',
'SERVER-WEBAPP GlobalFunctions.php access':'Information (NonCKC)',
'SERVER-WEBAPP DatabaseFunctions.php access':'Information (NonCKC)',
'SERVER-WEBAPP rolis guestbook remote file include attempt':'Delivery',
'SERVER-WEBAPP rolis guestbook access':'Information (NonCKC)',
'SERVER-WEBAPP friends.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_comment.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_edit.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_embed.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_help.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_license.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_logout.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_password.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_preview.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_settings.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_stats.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_templates_misc.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_templates.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_tpl_misc_new.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll admin_tpl_new.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll booth.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll poll_ssi.php access':'Information (NonCKC)',
'SERVER-WEBAPP Advanced Poll popup.php access':'Information (NonCKC)',
'SERVER-WEBAPP files.inc.php access':'Information (NonCKC)',
'SERVER-WEBAPP chatbox.php access':'Information (NonCKC)',
'SERVER-WEBAPP gallery remote file include attempt':'Delivery',
'SERVER-WEBAPP PayPal Storefront remote file include attempt':'Delivery',
'INDICATOR-COMPROMISE CVS non-relative path error response':'Information (NonCKC)',
'SERVER-OTHER CVS non-relative path access attempt':'Delivery',
'SERVER-OTHER ebola PASS overflow attempt':'Delivery',
'SERVER-OTHER ebola USER overflow attempt':'Delivery',
'SERVER-IIS foxweb.exe access':'Information (NonCKC)',
'SERVER-IIS foxweb.dll access':'Information (NonCKC)',
'SERVER-WEBAPP iSoft-Solutions QuickStore shopping cart quickstore.cgi access':'Information (NonCKC)',
'SERVER-IIS VP-ASP shopsearch.asp access':'Information (NonCKC)',
'SERVER-IIS VP-ASP ShopDisplayProducts.asp access':'Information (NonCKC)',
'SERVER-IIS sgdynamo.exe access':'Information (NonCKC)',
'SERVER-WEBAPP bsml.pl access':'Information (NonCKC)',
'SERVER-WEBAPP authentication_index.php access':'Information (NonCKC)',
'SERVER-MSSQL probe response overflow attempt':'Delivery',
'PROTOCOL-IMAP auth overflow attempt':'Delivery',
'SERVER-WEBAPP MatrikzGB privilege escalation attempt':'Delivery',
'PROTOCOL-FTP MKD format string attempt':'Delivery',
'PROTOCOL-FTP RENAME format string attempt':'Delivery',
'PROTOCOL-FTP Yak! FTP server default account login attempt':'Delivery',
'PROTOCOL-FTP RMD / attempt':'Delivery',
'PROTOCOL-TFTP PUT filename overflow attempt':'Delivery',
'PROTOCOL-FTP LIST buffer overflow attempt':'Delivery',
'PROTOCOL-TFTP NULL command attempt':'Delivery',
'PROTOCOL-FTP SITE CHMOD overflow attempt':'Delivery',
'SERVER-WEBAPP DCP-Portal remote file include editor script attempt':'Delivery',
'SERVER-WEBAPP DCP-Portal remote file include lib script attempt':'Delivery',
'PROTOCOL-FTP STOR overflow attempt':'Delivery',
'PROTOCOL-FTP XCWD overflow attempt':'Delivery',
'SERVER-WEBAPP PhpGedView search.php access':'Information (NonCKC)',
'SERVER-WEBAPP myPHPNuke chatheader.php access':'Information (NonCKC)',
'SERVER-WEBAPP myPHPNuke partner.php access':'Information (NonCKC)',
'SERVER-WEBAPP IdeaBox cord.php file include':'Information (NonCKC)',
'SERVER-WEBAPP IdeaBox notification.php file include':'Information (NonCKC)',
'SERVER-WEBAPP Invision Board emailer.php file include':'Information (NonCKC)',
'SERVER-WEBAPP WebChat db_mysql.php file include':'Information (NonCKC)',
'SERVER-WEBAPP WebChat english.php file include':'Information (NonCKC)',
'SERVER-WEBAPP Typo3 translations.php file include':'Information (NonCKC)',
'SERVER-WEBAPP Invision Board ipchat.php file include':'Information (NonCKC)',
'SERVER-WEBAPP myphpPagetool pt_config.inc file include':'Information (NonCKC)',
'SERVER-WEBAPP news.php file include':'Information (NonCKC)',
'SERVER-WEBAPP YaBB SE packages.php file include':'Information (NonCKC)',
'SERVER-WEBAPP Cyboards default_header.php access':'Information (NonCKC)',
'SERVER-WEBAPP Cyboards options_form.php access':'Information (NonCKC)',
'SERVER-WEBAPP newsPHP Language file include attempt':'Delivery',
'SERVER-WEBAPP PhpGedView PGV authentication_index.php base directory manipulation attempt':'Delivery',
'SERVER-WEBAPP PhpGedView PGV functions.php base directory manipulation attempt':'Delivery',
'SERVER-WEBAPP PhpGedView PGV config_gedcom.php base directory manipulation attempt':'Delivery',
'SERVER-WEBAPP ISAPISkeleton.dll access':'Information (NonCKC)',
'SERVER-WEBAPP BugPort config.conf file access':'Information (NonCKC)',
'SERVER-WEBAPP Sample_showcode.html access':'Information (NonCKC)',
'SERVER-WEBAPP Photopost PHP Pro showphoto.php access':'Information (NonCKC)',
'PROTOCOL-FTP XMKD overflow attempt':'Delivery',
'PROTOCOL-FTP NLST overflow attempt':'Delivery',
'MALWARE-CNC DoomJuice/mydoom.a backdoor upload/execute':'Exploitation',
'SERVER-OTHER ISAKMP first payload certificate request length overflow attempt':'Delivery',
'SERVER-OTHER ISAKMP second payload certificate request length overflow attempt':'Delivery',
'SERVER-OTHER ISAKMP third payload certificate request length overflow attempt':'Delivery',
'SERVER-OTHER ISAKMP forth payload certificate request length overflow attempt':'Delivery',
'SERVER-OTHER ISAKMP fifth payload certificate request length overflow attempt':'Delivery',
'SERVER-WEBAPP Checkpoint Firewall-1 HTTP parsing format string vulnerability attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB Session Setup NTLMSSP asn1 overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB-DS Session Setup NTLMSSP asn1 overflow attempt':'Delivery',
'SERVER-IIS NTLM ASN1 vulnerability scan attempt':'Reconaissance',
'SERVER-WEBAPP Apple QuickTime streaming server view_broadcast.cgi access':'Information (NonCKC)',
'PROTOCOL-FTP RNTO overflow attempt':'Delivery',
'PROTOCOL-FTP STOU overflow attempt':'Delivery',
'PROTOCOL-FTP APPE overflow attempt':'Delivery',
'PROTOCOL-FTP RETR overflow attempt':'Delivery',
'SERVER-WEBAPP /_admin access':'Information (NonCKC)',
'SERVER-WEBAPP Compaq web-based management agent denial of service attempt':'Delivery',
'SERVER-WEBAPP InteractiveQuery.jsp access':'Reconaissance',
'SERVER-WEBAPP CCBill whereami.cgi arbitrary command execution attempt':'Delivery',
'SERVER-WEBAPP CCBill whereami.cgi access':'Information (NonCKC)',
'SERVER-WEBAPP WAnewsletter newsletter.php file include attempt':'Delivery',
'SERVER-WEBAPP WAnewsletter db_type.php access':'Information (NonCKC)',
'SERVER-WEBAPP edittag.pl access':'Information (NonCKC)',
'NETBIOS SMB Session Setup andx username overflow attempt':'Delivery',
'NETBIOS SMB-DS Session Setup andx username overflow attempt':'Delivery',
'NETBIOS SMB Session Setup unicode username overflow attempt':'Delivery',
'NETBIOS SMB-DS Session Setup unicode andx username overflow attempt':'Delivery',
'SERVER-WEBAPP phptest.php access':'Information (NonCKC)',
'PROTOCOL-TELNET APC SmartSlot default admin account attempt':'Delivery',
'SERVER-WEBAPP util.pl access':'Information (NonCKC)',
'SERVER-WEBAPP Invision Power Board search.pl access':'Information (NonCKC)',
'PROTOCOL-POP APOP USER overflow attempt':'Delivery',
'SERVER-WEBAPP IGeneric Free Shopping Cart page.php access':'Information (NonCKC)',
'SERVER-WEBAPP RealNetworks RealSystem Server DESCRIBE buffer overflow attempt':'Delivery',
'INDICATOR-COMPROMISE successful cross site scripting forced download attempt':'Delivery',
'SERVER-OTHER ISAKMP delete hash with empty hash attempt':'Delivery',
'SERVER-OTHER ISAKMP initial contact notification without SPI attempt':'Delivery',
'SERVER-OTHER ISAKMP second payload initial contact notification without SPI attempt':'Delivery',
'PROTOCOL-FTP invalid MDTM command attempt':'Delivery',
'PROTOCOL-FTP format string attempt':'Delivery',
'POLICY-OTHER Microsoft Windows Terminal Server no encryption session initiation attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'PROTOCOL-NNTP sendsys overflow attempt':'Delivery',
'PROTOCOL-NNTP senduuname overflow attempt':'Delivery',
'PROTOCOL-NNTP version overflow attempt':'Delivery',
'PROTOCOL-NNTP checkgroups overflow attempt':'Delivery',
'PROTOCOL-NNTP ihave overflow attempt':'Delivery',
'PROTOCOL-NNTP sendme overflow attempt':'Delivery',
'PROTOCOL-NNTP newgroup overflow attempt':'Delivery',
'PROTOCOL-NNTP rmgroup overflow attempt':'Delivery',
'PROTOCOL-NNTP article post without path attempt':'Delivery',
'SERVER-WEBAPP MDaemon form2raw.cgi overflow attempt':'Delivery',
'SERVER-WEBAPP MDaemon form2raw.cgi access':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'FILE-MULTIMEDIA RealNetworks RealPlayer arbitrary javascript command attempt':'Delivery',
'FILE-MULTIMEDIA RealNetworks RealPlayer playlist file URL overflow attempt':'Delivery',
'FILE-MULTIMEDIA RealNetworks RealPlayer playlist http URL overflow attempt':'Delivery',
'FILE-MULTIMEDIA RealNetworks RealPlayer playlist rtsp URL overflow attempt':'Delivery',
'SERVER-WEBAPP NetObserve authentication bypass attempt':'Delivery',
'SERVER-OTHER ICQ SRV_MULTI/SRV_META_USER overflow attempt - ISS Witty Worm':'Delivery',
'SERVER-WEBAPP ServletManager access':'Information (NonCKC)',
'SERVER-WEBAPP setinfo.hts access':'Information (NonCKC)',
'PROTOCOL-FTP ALLO overflow attempt':'Delivery',
'POLICY-SOCIAL Yahoo IM successful logon':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM voicechat':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM ping':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM conference invitation':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM conference logon success':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM conference message':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo Messenger File Transfer Receive Request':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM message':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM successful chat join':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM conference offer invitation':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM conference request':'Information (NonCKC)',
'POLICY-SOCIAL Yahoo IM conference watch':'Information (NonCKC)',
'SERVER-OTHER Ethereal IGMP IGAP account overflow attempt':'Delivery',
'SERVER-OTHER Ethereal IGMP IGAP message overflow attempt':'Delivery',
'SERVER-OTHER Ethereal EIGRP prefix length overflow attempt':'Delivery',
'NETBIOS SMB-DS ADMIN$ share access':'Information (NonCKC)',
'SERVER-WEBAPP source.jsp access':'Information (NonCKC)',
'BROWSER-PLUGINS Symantec Norton Internet Security 2004 ActiveX clsid access':'Information (NonCKC)',
'SERVER-OTHER ISAKMP invalid identification payload attempt':'Delivery',
'SERVER-MAIL WinZip MIME content-type buffer overflow':'Exploitation',
'SERVER-MAIL WinZip MIME content-disposition buffer overflow':'Exploitation',
'SERVER-OTHER esignal STREAMQUOTE buffer overflow attempt':'Delivery',
'SERVER-OTHER esignal SNAPQUOTE buffer overflow attempt':'Delivery',
'OS-WINDOWS DCERPC NCACN-IP-TCP lsass DsRolerUpgradeDownlevelServer overflow attempt':'Delivery',
'OS-WINDOWS DCERPC NCADG-IP-UDP lsass DsRolerUpgradeDownlevelServer overflow attempt':'Delivery',
'SERVER-OTHER BGP spoofed connection reset attempt':'Delivery',
'SERVER-OTHER AFP FPLoginExt username buffer overflow attempt':'Delivery',
'PROTOCOL-FTP MDTM overflow attempt':'Delivery',
'SERVER-OTHER HP Web JetAdmin remote file upload attempt':'Delivery',
'SERVER-OTHER HP Web JetAdmin setinfo access':'Information (NonCKC)',
'SERVER-OTHER HP Web JetAdmin file write attempt':'Delivery',
'FILE-OTHER Nullsoft Winamp XM file buffer overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache GET overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache HEAD overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache PUT overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache POST overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache TRACE overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache DELETE overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache LOCK overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache MKCOL overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache COPY overflow attempt':'Delivery',
'SERVER-OTHER Oracle Web Cache MOVE overflow attempt':'Delivery',
'SERVER-OTHER rsync backup-dir directory traversal attempt':'Delivery',
'SERVER-WEBAPP McAfee ePO file upload attempt':'Delivery',
'NETBIOS NS lookup response name overflow attempt':'Delivery',
'NETBIOS NS lookup short response attempt':'Delivery',
'SERVER-WEBAPP modules.php access':'Information (NonCKC)',
'SERVER-WEBAPP PHPBB viewforum.php access':'Information (NonCKC)',
'SERVER-WEBAPP Emumail init.emu access':'Information (NonCKC)',
'SERVER-WEBAPP Emumail emumail.fcgi access':'Information (NonCKC)',
'SERVER-WEBAPP cPanel resetpass access':'Information (NonCKC)',
'SERVER-WEBAPP invalid HTTP version string':'Information (NonCKC)',
'SERVER-IIS SmarterTools SmarterMail frmGetAttachment.aspx access':'Information (NonCKC)',
'SERVER-IIS SmarterTools SmarterMail login.aspx buffer overflow attempt':'Delivery',
'SERVER-IIS SmarterTools SmarterMail frmCompose.asp access':'Information (NonCKC)',
'PROTOCOL-FTP RETR format string attempt':'Delivery',
'SERVER-WEBAPP Opt-X header.php remote file include attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.generate_replication_support buffer overflow attempt':'Delivery',
'FILE-OTHER local resource redirection attempt':'Delivery',
'SERVER-OTHER kerberos principal name overflow UDP':'Exploitation',
'SERVER-OTHER kerberos principal name overflow TCP':'Exploitation',
'SERVER-WEBAPP server negative Content-Length attempt':'Delivery',
'SERVER-WEBAPP SAP Crystal Reports crystalimagehandler.aspx access':'Information (NonCKC)',
'OS-WINDOWS SAP Crystal Reports crystalImageHandler.asp directory traversal attempt':'Delivery',
'SERVER-OTHER CVS Max-dotdot integer overflow attempt':'Delivery',
'SERVER-OTHER eMule buffer overflow attempt':'Delivery',
'SERVER-WEBAPP nessus 2.x 404 probe':'Information (NonCKC)',
'PUA-P2P eDonkey server response':'Information (NonCKC)',
'SERVER-WEBAPP TUTOS path disclosure attempt':'Delivery',
'OS-WINDOWS Microsoft Windows Content-Disposition CLSID command attempt':'Delivery',
'SERVER-WEBAPP Samba SWAT Authorization overflow attempt':'Delivery',
'SERVER-WEBAPP Samba SWAT Authorization port 901 overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_grouped_column buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_master_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.create_mview_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.compare_old_values buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE sysdbms_repcat_rgt.check_ddl_text buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.cancel_statistics buffer overflow attempt':'Delivery',
'SERVER-ORACLE LINK metadata buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_auth.revoke_surrogate_repcat buffer overflow attempt':'Delivery',
'SERVER-ORACLE time_zone buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_auth.grant_surrogate_repcat buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat.alter_mview_propagation buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_master_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_sna_utl.register_flavor_change buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_admin.unregister_user_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.send_old_values buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.repcat_import_check buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_admin.register_user_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_rectifier_diff.rectify buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_master_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_mview_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_instantiate.drop_site_instantiation buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla.ensure_not_published buffer overflow attempt':'Delivery',
'SERVER-ORACLE from_tz buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_instantiate.instantiate_offline buffer overflow attempt':'Delivery',
'SERVER-ORACLE Oracle 9i TNS Listener SERVICE_NAME Remote Buffer Overflow attempt':'Delivery',
'SERVER-ORACLE user name buffer overflow attempt':'Delivery',
'SERVER-ORACLE NUMTODSINTERVAL/NUMTOYMINTERVAL buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_og.begin_load buffer overflow attempt':'Delivery',
'SERVER-WEBAPP PHPNuke Forum viewtopic SQL insertion attempt':'Delivery',
'SERVER-OTHER HP Web JetAdmin ExecuteFile admin access':'Information (NonCKC)',
'SERVER-WEBAPP SSLv2 Client_Hello Challenge Length overflow attempt':'Delivery',
'SERVER-WEBAPP SSLv2 Client_Hello with pad Challenge Length overflow attempt':'Delivery',
'SERVER-WEBAPP Ipswitch WhatsUpGold instancename overflow attempt':'Delivery',
'PROTOCOL-IMAP login format string attempt':'Delivery',
'PROTOCOL-IMAP login literal format string attempt':'Delivery',
'PROTOCOL-POP PASS format string attempt':'Delivery',
'SERVER-IIS ping.asp access':'Information (NonCKC)',
'SERVER-WEBAPP processit access':'Information (NonCKC)',
'SERVER-WEBAPP ibillpm.pl access':'Information (NonCKC)',
'SERVER-WEBAPP pgpmail.pl access':'Information (NonCKC)',
'BROWSER-IE Microsoft Internet Explorer bitmap BitmapOffset integer overflow attempt':'Delivery',
'SERVER-WEBAPP sresult.exe access':'Information (NonCKC)',
'FILE-IMAGE libpng tRNS overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_delete_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_rgt.instantiate_offline buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_rgt.instantiate_online buffer overflow attempt':'Delivery',
'SERVER-ORACLE ctx_output.start_log buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_system.ksdwrt buffer overflow attempt':'Delivery',
'SERVER-ORACLE ctxsys.driddlr.subindexpopulate buffer overflow attempt':'Delivery',
'SERVER-ORACLE mdsys.sdo_admin.sdo_code_size buffer overflow attempt':'Delivery',
'SERVER-ORACLE mdsys.md2.validate_geom buffer overflow attempt':'Delivery',
'SERVER-ORACLE mdsys.md2.sdo_code_size buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.ltutil.pushdeferredtxns buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_rq.add_column buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_rectifier_diff.differences buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_internal_repcat.validate buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_internal_repcat.enable_receiver_trace buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_internal_repcat.disable_receiver_trace buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_defer_repcat.enable_propagation_to_dblink buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_defer_internal_sys.parallel_push_recovery buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_aqadm_sys.verify_queue_types buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_aqadm.verify_queue_types_no_queue buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_aqadm.verify_queue_types_get_nrp buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_aq_import_internal.aq_table_defn_update buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_utl.is_master buffer overflow attempt':'Delivery',
'SERVER-ORACLE alter file buffer overflow attempt':'Delivery',
'SERVER-ORACLE create file buffer overflow attempt':'Delivery',
'SERVER-ORACLE TO_CHAR buffer overflow attempt':'Delivery',
'SERVER-WEBAPP Oracle iSQLPlus sid overflow attempt':'Delivery',
'SERVER-WEBAPP Oracle iSQLPlus username overflow attempt':'Delivery',
'SERVER-WEBAPP Oracle iSQLPlus login.uix username overflow attempt':'Delivery',
'SERVER-WEBAPP Oracle 10g iSQLPlus login.unix connectID overflow attempt':'Delivery',
'FILE-IMAGE Microsoft Multiple Products JPEG parser heap overflow attempt':'Delivery',
'FILE-IMAGE JPEG parser multipacket heap overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_og.begin_flavor_change buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_og.begin_instantiation buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_og.end_flavor_change buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_og.end_instantiation buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_og.end_load buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_og.resume_subset_of_masters buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_snapshot.begin_load buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_offline_snapshot.end_load buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_rectifier_diff.differences buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_rectifier_diff.rectify buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.abort_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_column_group_to_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_columns_to_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_object_to_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_priority_char buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_priority_date buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_priority_nchar buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_priority_number buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_priority_nvarchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_priority_raw buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_priority_varchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_site_priority_site buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_unique_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.add_update_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_master_propagation buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_mview_propagation buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_priority_char buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_priority_date buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_priority_nchar buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_priority_number buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_priority_nvarchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_priority_raw buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_priority_varchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_site_priority_site buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_site_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.alter_snapshot_propagation buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_auth.revoke_surrogate_repcat buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.begin_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_column_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_delete_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_mview_repsites buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_priority_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_repsites buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_site_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_unique_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.comment_on_update_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.create_master_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.create_master_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.create_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.define_column_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.define_priority_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.define_site_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.do_deferred_repcat_admin buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_column_group_from_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_column_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_columns_from_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_delete_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_grouped_column buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_mview_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_object_from_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_priority_char buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_priority_date buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_priority_nchar buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_priority_number buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_priority_nvarchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_priority_raw buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_priority_varchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_site_priority_site buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_site_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_snapshot_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_unique_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.drop_update_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.execute_ddl buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.generate_replication_package buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_instantiate.instantiate_online buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.make_column_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.obsolete_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.publish_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.purge_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.purge_master_log buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.purge_statistics buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.refresh_mview_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.refresh_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.register_mview_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.register_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.register_statistics buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.relocate_masterdef buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.rename_shadow_column_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.resume_master_activity buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_rgt.check_ddl_text buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat_rgt.drop_site_instantiation buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.send_and_compare_old_values buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.set_columns buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.set_local_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.specify_new_masters buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.suspend_master_activity buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.unregister_mview_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.unregister_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.validate_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.validate_for_local_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla.abort_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla.add_object_to_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla.begin_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla.drop_object_from_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla_mas.add_column_group_to_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla_mas.add_columns_to_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla_mas.drop_column_group_from_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla_mas.drop_columns_from_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla_mas.obsolete_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla_mas.publish_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla_mas.purge_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla.set_local_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla.validate_flavor_definition buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_fla.validate_for_local_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.alter_master_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.comment_on_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.comment_on_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.create_master_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.create_master_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.do_deferred_repcat_admin buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.drop_master_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.generate_replication_package buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.purge_master_log buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.relocate_masterdef buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.rename_shadow_column_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.resume_master_activity buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_mas.suspend_master_activity buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.alter_snapshot_propagation buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.create_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.drop_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.drop_snapshot_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.refresh_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.register_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.repcat_import_check buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.unregister_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_utl4.drop_master_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_utl.drop_an_object buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.create_mview_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.create_snapshot_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.generate_mview_support buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.generate_replication_trigger buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.generate_snapshot_support buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.remove_master_databases buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.switch_mview_master buffer overflow attempt':'Delivery',
'SERVER-ORACLE dbms_repcat.switch_snapshot_master buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_delete_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_priority_char buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_priority_date buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_priority_nchar buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_priority_number buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_priority_nvarchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_priority_raw buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_priority_varchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_site_priority_site buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_unique_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.add_update_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_priority_char buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_priority_date buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_priority_nchar buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_priority_number buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_priority_nvarchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_priority_raw buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_priority_varchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_site_priority_site buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.alter_site_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.cancel_statistics buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.comment_on_delete_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.comment_on_priority_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.comment_on_site_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.comment_on_unique_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.comment_on_update_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.define_priority_group buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.define_site_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_delete_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_priority_char buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_priority_date buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_priority_nchar buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_priority_number buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_priority_nvarchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_priority_raw buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_priority_varchar2 buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_site_priority_site buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_site_priority buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_unique_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.drop_update_resolution buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.purge_statistics buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_conf.register_statistics buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.alter_snapshot_propagation buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.create_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.create_snapshot_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.create_snapshot_repschema buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.drop_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.drop_snapshot_repobject buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.drop_snapshot_repschema buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.generate_snapshot_support buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.refresh_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.refresh_snapshot_repschema buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.register_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.repcat_import_check buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.set_local_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.switch_snapshot_master buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.unregister_snapshot_repgroup buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna_utl.switch_snapshot_master buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_sna.validate_for_local_flavor buffer overflow attempt':'Delivery',
'SERVER-ORACLE sys.dbms_repcat_untrusted.register_snapshot_repgroup buffer overflow attempt':'Delivery',
'PROTOCOL-DNS UDP inverse query':'Reconaissance',
'PROTOCOL-DNS TCP inverse query':'Reconaissance',
'NETBIOS SMB repeated logon failure':'Information (NonCKC)',
'NETBIOS SMB-DS repeated logon failure':'Information (NonCKC)',
'SERVER-WEBAPP PhpGedView PGV base directory manipulation':'Information (NonCKC)',
'OS-WINDOWS Microsoft Windows XPAT pattern overflow attempt':'Delivery',
'OS-WINDOWS DCERPC NCACN-IP-TCP nddeapi NDdeSetTrustedShareW overflow attempt':'Delivery',
'NETBIOS DCERPC NCACN-IP-TCP winreg InitiateSystemShutdown attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB Session Setup NTLMSSP unicode asn1 overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB Session Setup NTLMSSP andx asn1 overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB Session Setup NTLMSSP unicode andx asn1 overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB-DS Session Setup NTLMSSP unicode asn1 overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB-DS Session Setup NTLMSSP andx asn1 overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB-DS Session Setup NTLMSSP unicode andx asn1 overflow attempt':'Delivery',
'SERVER-OTHER Volition Freespace 2 buffer overflow attempt':'Delivery',
'PROTOCOL-IMAP command overflow attempt':'Delivery',
'PROTOCOL-IMAP delete literal overflow attempt':'Delivery',
'MALWARE-BACKDOOR NetBus Pro 2.0 connection request':'Exploitation',
'MALWARE-CNC RUX the Tick get windows directory':'Exploitation',
'MALWARE-CNC RUX the Tick get system directory':'Exploitation',
'MALWARE-CNC RUX the Tick upload/execute arbitrary file':'Exploitation',
'MALWARE-CNC Asylum 0.1 connection request':'Exploitation',
'MALWARE-CNC Asylum 0.1 connection':'Exploitation',
'MALWARE-CNC Insane Network 4.0 connection':'Exploitation',
'MALWARE-CNC Insane Network 4.0 connection port 63536':'Exploitation',
'OS-WINDOWS Microsoft Windows WINS overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE oversized Security Descriptor attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE andx oversized Security Descriptor attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode oversized Security Descriptor attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode andx oversized Security Descriptor attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE oversized Security Descriptor attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE andx oversized Security Descriptor attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode oversized Security Descriptor attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode andx oversized Security Descriptor attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE SACL overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE andx SACL overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode SACL overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode andx SACL overflow attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE SACL overflow attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE andx SACL overflow attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode SACL overflow attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode andx SACL overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE DACL overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE andx DACL overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode DACL overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode andx DACL overflow attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE DACL overflow attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE andx DACL overflow attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode DACL overflow attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode andx DACL overflow attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE andx invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode andx invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE andx invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode andx invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE andx invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB NT Trans NT CREATE unicode andx invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE andx invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode invalid SACL ace size dos attempt':'Delivery',
'NETBIOS SMB-DS NT Trans NT CREATE unicode andx invalid SACL ace size dos attempt':'Delivery',
'PROTOCOL-IMAP copy literal overflow attempt':'Delivery',
'APP-DETECT distccd remote command execution attempt':'Delivery',
'SERVER-WEBAPP NetScreen SA 5000 delhomepage.cgi access':'Information (NonCKC)',
'MALWARE-BACKDOOR Vampire 1.2 connection request':'Exploitation',
'MALWARE-BACKDOOR Vampire 1.2 connection confirmation':'Exploitation',
'PROTOCOL-IMAP append literal overflow attempt':'Delivery',
'PROTOCOL-IMAP append overflow attempt':'Delivery',
'PROTOCOL-IMAP examine literal overflow attempt':'Delivery',
'PROTOCOL-IMAP fetch literal overflow attempt':'Delivery',
'PROTOCOL-IMAP fetch overflow attempt':'Delivery',
'PROTOCOL-IMAP status literal overflow attempt':'Delivery',
'PROTOCOL-IMAP status overflow attempt':'Delivery',
'PROTOCOL-IMAP SUBSCRIBE literal overflow attempt':'Delivery',
'PROTOCOL-IMAP SUBSCRIBE overflow attempt':'Delivery',
'PROTOCOL-IMAP unsubscribe literal overflow attempt':'Delivery',
'PROTOCOL-IMAP UNSUBSCRIBE overflow attempt':'Delivery',
'PROTOCOL-FTP RNFR overflow attempt':'Delivery',
'PROTOCOL-NNTP Microsoft Windows SEARCH pattern overflow attempt':'Delivery',
'BROWSER-IE Microsoft Internet Explorer ANI file parsing buffer overflow attempt':'Delivery',
'SERVER-OTHER Unreal Tournament secure overflow attempt':'Delivery',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'SERVER-OTHER Veritas backup overflow attempt':'Delivery',
'SERVER-OTHER AOL Instant Messenger goaway message buffer overflow attempt':'Delivery',
'SERVER-WEBAPP 3Com 3CRADSL72 ADSL 11g Wireless Router app_sta.stm access attempt':'Delivery',
'SERVER-IIS w3who.dll buffer overflow attempt':'Delivery',
'FILE-MULTIMEDIA Nullsoft Winamp cda file name overflow attempt':'Delivery',
'SERVER-OTHER squid WCCP I_SEE_YOU message overflow attempt':'Delivery',
'OS-WINDOWS DCERPC NCACN-IP-TCP llsrpc LlsrConnect overflow attempt':'Delivery',
'PUA-OTHER Microsoft MSN Messenger png overflow':'Exploitation',
'SERVER-WEBAPP mailman directory traversal attempt':'Delivery',
'FILE-IMAGE Microsoft and libpng multiple products PNG large image width overflow attempt':'Delivery',
'FILE-IMAGE Microsoft Multiple Products PNG large image height download attempt':'Delivery',
'FILE-IMAGE Microsoft PNG large colour depth download attempt':'Delivery',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'OS-WINDOWS Microsoft Windows SMB Trans2 FIND_FIRST2 command response overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB Trans2 FIND_FIRST2 response andx overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB-DS Trans2 FIND_FIRST2 response overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows SMB-DS Trans2 FIND_FIRST2 response andx overflow attempt':'Delivery',
'PROTOCOL-TELNET login buffer overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows HTML Help hhctrl.ocx clsid access attempt':'Delivery',
'BROWSER-IE Microsoft Internet Explorer 5/6 object type overflow attempt':'Delivery',
'SERVER-IIS SQLXML content type overflow':'Exploitation',
'PROTOCOL-FINGER / execution attempt':'Delivery',
'SQL sa brute force failed login attempt':'Delivery',
'PROTOCOL-DNS TCP inverse query overflow':'Exploitation',
'PROTOCOL-DNS UDP inverse query overflow':'Exploitation',
'MALWARE-BACKDOOR BackOrifice 2000 Inbound Traffic':'Exploitation',
'OS-WINDOWS DCERPC NCACN-IP-TCP ISystemActivator CoGetInstanceFromFile attempt':'Delivery',
'OS-WINDOWS DCERPC NCADG-IP-UDP ISystemActivator CoGetInstanceFromFile attempt':'Delivery',
'OS-WINDOWS DCERPC NCADG-IP-UDP msqueue function 4 overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows Media Player directory traversal via Content-Disposition attempt':'Delivery',
'SERVER-IIS .cmd executable file parsing attack':'Information (NonCKC)',
'SERVER-IIS .bat executable file parsing attack':'Information (NonCKC)',
'OS-WINDOWS name query overflow attempt TCP':'Delivery',
'OS-WINDOWS name query overflow attempt UDP':'Delivery',
'OS-WINDOWS Microsoft Windows WINS name query overflow attempt TCP':'Delivery',
'OS-WINDOWS Microsoft Windows WINS name query overflow attempt UDP':'Delivery',
'SERVER-IIS httpodbc.dll access - nimda':'Information (NonCKC)',
'OS-WINDOWS DCERPC NCACN-IP-TCP winreg OpenKey overflow attempt':'Delivery',
'OS-WINDOWS Messenger message little endian overflow attempt':'Delivery',
'OS-WINDOWS Messenger message overflow attempt':'Delivery',
'OS-WINDOWS DCERPC NCACN-IP-TCP irot IrotIsRunning/Revoke overflow attempt':'Delivery',
'OS-WINDOWS DCERPC NCADG-IP-UDP irot IrotIsRunning/Revoke overflow attempt':'Delivery',
'SQL sa brute force failed login unicode attempt':'Delivery',
'PROTOCOL-TELNET login buffer non-evasive overflow attempt':'Delivery',
'OS-WINDOWS DCERPC NCACN-IP-TCP ISystemActivator RemoteCreateInstance attempt':'Delivery',
'OS-WINDOWS DCERPC NCADG-IP-UDP ISystemActivator RemoteCreateInstance attempt':'Delivery',
'OS-WINDOWS DCERPC NCACN-IP-TCP IActivation remoteactivation overflow attempt':'Delivery',
'OS-WINDOWS Microsoft Windows TCP print service overflow attempt':'Delivery',
'SERVER-OTHER Arkeia client backup system info probe':'Information (NonCKC)',
'SERVER-OTHER Arkeia client backup generic info probe':'Information (NonCKC)',
'SERVER-OTHER Bontago Game Server Nickname buffer overflow':'Exploitation',
'SERVER-MYSQL 4.0 root login attempt':'Delivery',
'SERVER-OTHER Arkeia backup client type 77 overflow attempt':'Delivery',
'SERVER-OTHER Arkeia backup client type 84 overflow attempt':'Delivery',
'PUA-P2P Manolito Search Query':'Reconaissance',
'PROTOCOL-FTP REST with numeric argument':'Information (NonCKC)',
'SERVER-MAIL Content-Type overflow attempt':'Delivery',
'BROWSER-IE Microsoft Internet Explorer Content-Encoding overflow attempt':'Delivery',
'SERVER-WEBAPP awstats access':'Information (NonCKC)',
'SERVER-WEBAPP awstats.pl command execution attempt':'Delivery',
'PROTOCOL-VOIP inbound INVITE message':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SERVER-OTHER multiple products blacknurse ICMP denial of service attempt':'Reconaissance',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'BLACKLIST User-Agent known malicious user-agent string DataCha0s':'Information (NonCKC)',
'established,to_client':'Information (NonCKC)',
'established,to_client':'Information (NonCKC)',
'BLACKLIST URI - known scanner tool muieblackcat':'Reconaissance',
'BLACKLIST User-Agent known malicious user-agent string Morfeus Scanner':'Reconaissance',
'POLICY-OTHER TRENDnet IP Camera anonymous access attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'SERVER-WEBAPP Remote Execution Backdoor Attempt Against Horde':'Delivery',
'FILE-PDF hostile PDF associated with Laik exploit kit':'Information (NonCKC)',
'EXPLOIT-KIT Blackhole exploit kit JavaScript carat string splitting with hostile applet':'Information (NonCKC)',
'MALWARE-CNC URI request for known malicious URI - base64 encoded':'Exploitation',
'MALWARE-CNC Win.Trojan.TDSS variant outbound connection':'Exploitation',
'to_server,':'Information (NonCKC)',
'EXPLOIT-KIT Blackhole exploit kit landing page with specific structure - prototype catch':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'EXPLOIT-KIT Blackhole exploit kit landing page with specific structure - prototype catch':'Information (NonCKC)',
'SERVER-WEBAPP System variable directory traversal attempt - %ALLUSERSPROFILE%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %PROGRAMDATA%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %APPDATA%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %COMMONPROGRAMFILES%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %COMMONPROGRAMFILES - x86%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %COMSPEC%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %HOMEDRIVE%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %HOMEPATH%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %LOCALAPPDATA%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %PROGRAMFILES%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %PROGRAMFILES - X86%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %SystemDrive%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %SystemRoot%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %TEMP%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %TMP%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %USERDATA%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %USERNAME%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %USERPROFILE%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %WINDIR%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %PUBLIC%':'Delivery',
'SERVER-WEBAPP System variable directory traversal attempt - %PSModulePath%':'Delivery',
'SERVER-WEBAPP System variable in URI attempt - %COMPUTERNAME%':'Delivery',
'SERVER-WEBAPP System variable in URI attempt - %LOGONSERVER%':'Delivery',
'SERVER-WEBAPP System variable in URI attempt - %PATH%':'Delivery',
'SERVER-WEBAPP System variable in URI attempt - %PATHEXT%':'Delivery',
'SERVER-WEBAPP System variable in URI attempt - %PROMPT%':'Delivery',
'SERVER-WEBAPP System variable in URI attempt - %USERDOMAIN%':'Delivery',
'to_client,established':'Information (NonCKC)',
'MALWARE-CNC TDS Sutra - request in.cgi':'Exploitation',
'MALWARE-OTHER TDS Sutra - page redirecting to a SutraTDS':'Exploitation',
'MALWARE-OTHER TDS Sutra - HTTP header redirecting to a SutraTDS':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'SERVER-WEBAPP PHP-CGI remote file include attempt':'Delivery',
'BLACKLIST DNS request for known malware domain slade.safehousenumber.com - Mal/Rimecud-R':'Exploitation',
'BLACKLIST DNS request for known malware domain portal.roomshowerbord.com - Mal/EncPk-ADU':'Exploitation',
'INDICATOR-COMPROMISE script before DOCTYPE possible malicious redirect attempt':'Delivery',
'INDICATOR-OBFUSCATION hex escaped characters in setTimeout call':'Information (NonCKC)',
'INDICATOR-OBFUSCATION hex escaped characters in addEventListener call':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'INDICATOR-OBFUSCATION known packer routine with secondary obfuscation':'Information (NonCKC)',
'INDICATOR-OBFUSCATION JavaScript built-in function parseInt appears obfuscated - likely packer or encoder':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'BLACKLIST DNS request for known malware domain api.wipmania.com - Troj.Dorkbot-AO':'Exploitation',
'BLACKLIST DNS request for known malware domain lolcantpwnme.net - W32.DorkBot-S':'Exploitation',
'BLACKLIST DNS request for known malware domain rewt.ru - W32.DorkBot-S':'Exploitation',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'OS-MOBILE Android/Fakelash.A!tr.spy trojan command and control channel traffic':'Information (NonCKC)',
'INDICATOR-COMPROMISE IP only webpage redirect attempt':'Delivery',
'INDICATOR-COMPROMISE IP only webpage redirect attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'POLICY-SPAM 1.usa.gov URL in email, possible spam redirect':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Zeus variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'APP-DETECT Acunetix web vulnerability scanner probe attempt':'Reconaissance',
'APP-DETECT Acunetix web vulnerability scanner authentication attempt':'Reconaissance',
'APP-DETECT Acunetix web vulnerability scanner RFI attempt':'Reconaissance',
'APP-DETECT Acunetix web vulnerability scanner base64 XSS attempt':'Delivery',
'APP-DETECT Acunetix web vulnerability scanner URI injection attempt':'Reconaissance',
'APP-DETECT Acunetix web vulnerability scanner prompt XSS attempt':'Delivery',
'APP-DETECT Acunetix web vulnerability scanner XSS attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'OS-MOBILE Apple iPod User-Agent detected':'Information (NonCKC)',
'OS-MOBILE Apple iPad User-Agent detected':'Information (NonCKC)',
'OS-MOBILE Apple iPhone User-Agent detected':'Information (NonCKC)',
'OS-MOBILE Android User-Agent detected':'Information (NonCKC)',
'OS-MOBILE Nokia User-Agent detected':'Information (NonCKC)',
'OS-MOBILE Samsung User-Agent detected':'Information (NonCKC)',
'OS-MOBILE Kindle User-Agent detected':'Information (NonCKC)',
'OS-OTHER Nintendo User-Agent detected':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Zeus variant outbound connection - MSIE7 No Referer No Cookie':'Exploitation',
'APP-DETECT Ammyy remote access tool':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Zeus variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Proxyier variant outbound connection':'Exploitation',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'APP-DETECT Absolute Software Computrace outbound connection - search.dnssearch.org':'Information (NonCKC)',
'APP-DETECT Absolute Software Computrace outbound connection - search.namequery.com':'Reconaissance',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Daws Trojan Outbound Plaintext over SSL Port':'Exploitation',
'MALWARE-CNC file path used as User-Agent - potential Trojan':'Exploitation',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC OSX.Trojan.Flashfake variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE IP address check to dyndns.org detected':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-OTHER UTF-8 BOM in zip file attachment detected':'Exploitation',
'MALWARE-OTHER UTF-8 BOM in zip file attachment detected':'Exploitation',
'MALWARE-OTHER UTF-8 BOM in zip file attachment detected':'Exploitation',
'APP-DETECT Ufasoft bitcoin miner possible data upload':'Information (NonCKC)',
'BLACKLIST DNS request for known malware domain suppp.cantvenlinea.biz - Bitcoin Miner upload':'Exploitation',
'MALWARE-CNC Win.Trojan.Gamarue variant outbound connection':'Exploitation',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'INDICATOR-COMPROMISE IP address check to j.maxmind.com detected':'Information (NonCKC)',
'MALWARE-OTHER Win.Worm.Dorkbot folder snkb0ptz creation attempt SMB':'Exploitation',
'MALWARE-OTHER Win.Worm.Dorkbot executable snkb0ptz.exe creation attempt SMB':'Exploitation',
'MALWARE-OTHER Win.Worm.Dorkbot Desktop.ini snkb0ptz.exe creation attempt SMB':'Exploitation',
'MALWARE-CNC Win.Trojan.Magic variant inbound connection':'Exploitation',
'SERVER-ORACLE Oracle WebCenter FatWire Satellite Server header injection on blobheadername2 attempt':'Delivery',
'SERVER-ORACLE Oracle WebCenter FatWire Satellite Server header injection on blobheadername2 attempt':'Delivery',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Unknown Thinner Encrypted POST botnet C&C':'Exploitation',
'SERVER-WEBAPP JavaScript tag in User-Agent field possible XSS attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Unix.Backdoor.Cdorked redirect attempt':'Delivery',
'MALWARE-CNC Unknown malware - Incorrect headers - Referer HTTP/1.0':'Exploitation',
'PUA-ADWARE Win.Adware.BProtector browser hijacker dll list download attempt':'Delivery',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Bancos variant outbound connection - getcomando POST data':'Exploitation',
'MALWARE-CNC Win.Trojan.Zeus variant outbound connection':'Exploitation',
'EXPLOIT-KIT Nuclear exploit kit Spoofed Host Header .com- requests':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Potential hostile executable served from compromised or malicious WordPress site attempt':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'INDICATOR-COMPROMISE config.inc.php in iframe':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'BROWSER-WEBKIT Possible Google Chrome Plugin install from non-trusted source':'Information (NonCKC)',
'BROWSER-FIREFOX Possible Mozilla Firefox Plugin install from non-Mozilla source':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'EXPLOIT-KIT Blackholev2 exploit kit Initial Gate from Linked-In Mailing Campaign':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC RDN Banker POST variant outbound connection':'Exploitation',
'MALWARE-CNC RDN Banker Strange Google Traffic':'Exploitation',
'to_client,established':'Information (NonCKC)',
'EXPLOIT-KIT Blackholev2 exploit kit Initial Gate from NatPay Mailing Campaign':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC ZeroAccess Encrypted 128-byte POST No Accept Headers':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SQL generic convert injection attempt - GET parameter':'Delivery',
'EXPLOIT-KIT DotkaChef/Rmayana/DotCache exploit kit inbound java exploit download':'Information (NonCKC)',
'EXPLOIT-KIT DotkaChef/Rmayana/DotCache exploit kit inbound java exploit download':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Gozi Data Theft POST Data':'Exploitation',
'MALWARE-CNC Win.Trojan.Gozi Trojan Data Theft POST URL':'Exploitation',
'MALWARE-CNC Win.Trojan.Pirminay variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Dapato variant inbound response connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'BLACKLIST User-Agent known malicious user-agent string pb - Htbot':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Blocker Download':'Exploitation',
'INDICATOR-COMPROMISE Unknown ?1 redirect':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'BLACKLIST DNS request for known malware domain scari-elegante.ro - Yakes Trojan':'Exploitation',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Meredrop variant outbound connection GET Request':'Exploitation',
'MALWARE-CNC Win.Trojan.Meredrop variant outbound connection POST Request':'Exploitation',
'to_server,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Apache auto_prepend_file a.control.bin C2 traffic':'Information (NonCKC)',
'MALWARE-CNC Potential Bancos Brazilian Banking Trojan Browser Proxy Autoconfig File':'Exploitation',
'to_client,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.ZeroAccess 111-byte URL variant outbound connection':'Exploitation',
'MALWARE-CNC Win.Trojan.Cridex Encrypted POST w/ URL Pattern':'Exploitation',
'MALWARE-CNC Yakes Trojan HTTP Header Structure':'Exploitation',
'INDICATOR-COMPROMISE All Numbers .EXE file name from abnormally ordered HTTP headers - Potential Yakes Trojan Download':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Kryptik Drive-by Download Malware':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'MALWARE-OTHER self-signed SSL certificate with default MyCompany Ltd organization name':'Exploitation',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'MALWARE-CNC Win.Ransomware.Urausy outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.PRISM variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'PROTOCOL-VOIP Possible SIP OPTIONS service information gathering attempt':'Delivery',
'PROTOCOL-VOIP Excessive number of SIP 4xx responses potential user or password guessing attempt':'Delivery',
'PROTOCOL-VOIP Ghost call attack attempt':'Delivery',
'PROTOCOL-VOIP Possible SIP OPTIONS service information gathering attempt':'Delivery',
'PROTOCOL-VOIP Ghost call attack attempt':'Delivery',
'PROTOCOL-VOIP Excessive number of SIP 4xx responses potential user or password guessing attempt':'Delivery',
'EXPLOIT-KIT Blackholev2/Cool exploit kit payload download attempt':'Delivery',
'PUA-ADWARE Vittalia adware - get ads':'Information (NonCKC)',
'PUA-ADWARE Vittalia adware - post install':'Information (NonCKC)',
'PUA-ADWARE Vittalia adware outbound connection - pre install':'Information (NonCKC)',
'PUA-TOOLBARS Vittalia adware outbound connection - Eazel toolbar install':'Information (NonCKC)',
'PUA-TOOLBARS Vittalia adware outbound connection - offers':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Backdoor.Chopper web shell connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-OTHER Win.Trojan.Kuluoz outbound download request':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'EXPLOIT-KIT Blackholev2 exploit kit landing page':'Information (NonCKC)',
'EXPLOIT-KIT Blackholev2/Cool exploit kit exploit download attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Napolar data theft':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /default.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /file.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /home.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /install.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /login.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /search.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /start.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /welcome.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /index.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /setup.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Conficker variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Foreign variant outbound connection - /html2/':'Exploitation',
'MALWARE-CNC Win.Trojan.Foreign variant outbound connection - MSIE 7.1':'Exploitation',
'MALWARE-CNC Win.Trojan.Foreign variant outbound connection - MSIE 7.2':'Exploitation',
'PUA-ADWARE Linkury outbound time check':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Kuluoz Potential Phishing URL':'Exploitation',
'BLACKLIST DNS request for known malware domain- Win.Vobfus worm variant':'Exploitation',
'SERVER-WEBAPP vBulletin upgrade.php exploit attempt':'Delivery',
'EXPLOIT-KIT Blackholev2/Cool exploit kit payload download attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'EXPLOIT-KIT Blackholev2/Cool exploit kit exploit download attempt':'Delivery',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'INDICATOR-OBFUSCATION large number of calls to chr function - possible sql injection obfuscation':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Kazy variant outbound connection':'Exploitation',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Asprox/Kuluoz variant connection':'Exploitation',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'INDICATOR-SCAN inbound probing for IPTUX messenger port ':'Reconaissance',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /main.htm GET Encrypted Payload':'Exploitation',
'MALWARE-CNC Win.Trojan.Fareit variant outbound connection - /online.htm GET Encrypted Payload':'Exploitation',
'MALWARE-OTHER SQL Slammer worm propagation attempt inbound':'Exploitation',
'PROTOCOL-DNS DNS query amplification attempt':'Delivery',
'PROTOCOL-DNS Malformed DNS query with HTTP content':'Reconaissance',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'INDICATOR-COMPROMISE potential malware download - single digit .exe file download':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Zeus variant outbound connection - MSIE7 No Referer No Cookie':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-BACKDOOR Zollard variant outbound connection attempt':'Exploitation',
'MALWARE-CNC Win.Trojan.Symmi variant network connectivity check':'Exploitation',
'MALWARE-CNC Win.Trojan.Symmi variant network connectivity check':'Exploitation',
'to_server,established':'Information (NonCKC)',
'BLACKLIST DNS request for known malware domain appropriations.co.cc':'Exploitation',
'BLACKLIST DNS request for known malware domain havingbeothers.co.cc':'Exploitation',
'to_server,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE exe.exe download':'Information (NonCKC)',
'BLACKLIST DNS reverse lookup response to malicious domain .dataclub.biz - Win.Trojan.Bunitu.G':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Worm.Steckt IRCbot executable download':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'EXPLOIT-KIT CritX exploit kit payload download attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Dropper inbound encrypted traffic':'Exploitation',
'MALWARE-CNC Win.Trojan.Dropper outbound encrypted traffic - potential exfiltration':'Exploitation',
'MALWARE-CNC Win.Trojan.Dropper outbound encrypted traffic':'Exploitation',
'MALWARE-CNC Win.Trojan.Zeus variant outbound connection':'Exploitation',
'POLICY-SPAM Potential phishing attack - .zip receipt filename download with .exe name within .zip the same':'Information (NonCKC)',
'POLICY-SPAM Potential phishing attack - .zip shipping filename download with .exe name within .zip the same':'Information (NonCKC)',
'POLICY-SPAM Potential phishing attack - .zip voicemail filename download with .exe name within .zip the same':'Information (NonCKC)',
'POLICY-SPAM Potential phishing attack - .zip statement filename download with .exe name within .zip the same':'Information (NonCKC)',
'PROTOCOL-ICMP Unusual L3retriever Ping detected':'Reconaissance',
'PROTOCOL-ICMP Unusual Microsoft Windows Ping detected':'Reconaissance',
'PROTOCOL-ICMP Unusual PING detected':'Reconaissance',
'PROTOCOL-ICMP Unusual Microsoft Windows 7 Ping detected':'Reconaissance',
'MALWARE-CNC Win.Trojan.Fexel variant outbound connection':'Exploitation',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,':'Information (NonCKC)',
'to_server,':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Pirminay variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'BLACKLIST DNS request for known malware domain 0zz0.com - Win.Trojan.Napolar':'Exploitation',
'to_server':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Pushdo variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Androm variant outbound connection':'Exploitation',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Strictor variant outbound connection attempt':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Linux.Trojan.Calfbot outbound connection':'Exploitation',
'to_server':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established,only_stream':'Information (NonCKC)',
'to_client,established,only_stream':'Information (NonCKC)',
'to_client,established,only_stream':'Information (NonCKC)',
'to_client,established,only_stream':'Information (NonCKC)',
'to_server,established,only_stream':'Information (NonCKC)',
'to_server,established,only_stream':'Information (NonCKC)',
'to_server,established,only_stream':'Information (NonCKC)',
'to_server,established,only_stream':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Ramdo variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Linux.Trojan.Elknot outbound connection':'Exploitation',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Zeus variant outbound connection':'Exploitation',
'to_server':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Potential malware download - .doc.exe within .zip file':'Exploitation',
'INDICATOR-COMPROMISE Potential malware download - .gif.exe within .zip file':'Exploitation',
'INDICATOR-COMPROMISE Potential malware download - .jpeg.exe within .zip file':'Exploitation',
'INDICATOR-COMPROMISE Potential malware download - .jpg.exe within .zip file':'Exploitation',
'INDICATOR-COMPROMISE Potential malware download - .pdf.exe within .zip file':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.SpyBanker variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Necurs variant outbound connection':'Exploitation',
'MALWARE-CNC Win.Trojan.Kuluoz outbound connection':'Exploitation',
'to_client,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Symmi outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Dyre publickey outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SERVER-APACHE Apache Chunked-Encoding worm attempt':'Delivery',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.ChoHeap variant outbound connection':'Exploitation',
'MALWARE-CNC Win.Trojan.ChoHeap variant outbound connection':'Exploitation',
'EXPLOIT-KIT Rig Exploit Kit Outbound DGA Request':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Badur download attempt':'Exploitation',
'to_server,established':'Information (NonCKC)',
'FILE-IMAGE Microsoft Multiple Products JPEG parser heap overflow attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'EXPLOIT-KIT Astrum exploit kit landing page':'Information (NonCKC)',
'EXPLOIT-KIT Astrum exploit kit payload delivery':'Information (NonCKC)',
'EXPLOIT-KIT Astrum exploit kit payload delivery':'Information (NonCKC)',
'EXPLOIT-KIT Astrum exploit kit redirection attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'EXPLOIT-KIT Astrum exploit kit payload delivery':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'stateless':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'OS-OTHER Bash environment variable injection attempt':'Delivery',
'OS-OTHER Bash environment variable injection attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'BROWSER-FIREFOX Mozilla 1.0 Javascript arbitrary cookie access attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'OS-OTHER Bash CGI environment variable injection attempt':'Delivery',
'OS-OTHER Bash CGI environment variable injection attempt':'Delivery',
'OS-OTHER Bash environment variable injection attempt':'Delivery',
'MALWARE-CNC Win.Trojan.GameOverZeus variant outbound connection':'Exploitation',
'SERVER-OTHER AOL Instant Messenger goaway message buffer overflow attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Potential malware download - _pdf.exe within .zip file':'Exploitation',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Wiper variant outbound connection':'Exploitation',
'to_server,':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'APP-DETECT Absolute Software Computrace outbound connection - 209.53.113.223':'Information (NonCKC)',
'APP-DETECT Absolute Software Computrace outbound connection - absolute.com':'Information (NonCKC)',
'APP-DETECT Absolute Software Computrace outbound connection - bh.namequery.com':'Reconaissance',
'APP-DETECT Absolute Software Computrace outbound connection - namequery.nettrace.co.za':'Reconaissance',
'APP-DETECT Absolute Software Computrace outbound connection - search.us.namequery.com':'Reconaissance',
'APP-DETECT Absolute Software Computrace outbound connection - search2.namequery.com':'Reconaissance',
'APP-DETECT Absolute Software Computrace outbound connection - search64.namequery.com':'Reconaissance',
'to_server,':'Information (NonCKC)',
'to_server,':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'FILE-IMAGE Microsoft and libpng multiple products PNG large image width overflow attempt':'Delivery',
'MALWARE-BACKDOOR Win.Trojan.Wiper inbound communication attempt':'Exploitation',
'MALWARE-BACKDOOR Win.Trojan.Wiper outbound communication attempt':'Exploitation',
'MALWARE-BACKDOOR Win.Trojan.Wiper download attempt':'Exploitation',
'MALWARE-BACKDOOR Win.Trojan.Wiper download attempt':'Exploitation',
'MALWARE-BACKDOOR Win.Trojan.Wiper inbound communication attempt':'Exploitation',
'MALWARE-BACKDOOR Win.Trojan.Wiper outbound communication attempt':'Exploitation',
'MALWARE-BACKDOOR Win.Trojan.Wiper inbound communication attempt':'Exploitation',
'MALWARE-BACKDOOR Win.Trojan.Wiper download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper listener download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper download attempt':'Exploitation',
'MALWARE-OTHER Win.Trojan.Wiper download attempt':'Exploitation',
'MALWARE-TOOLS Win.Trojan.Wiper proxy tools download attempt':'Exploitation',
'MALWARE-TOOLS Win.Trojan.Wiper proxy communication attempt':'Exploitation',
'MALWARE-TOOLS Win.Trojan.Wiper proxy tool download attempt':'Exploitation',
'to_server,':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,':'Information (NonCKC)',
'to_server,':'Information (NonCKC)',
'to_server,':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Win.Trojan.Blocker variant outbound connection attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'FILE-IMAGE Microsoft emf file download request':'Information (NonCKC)',
'PUA-ADWARE Adware Goobzo/CrossRider variant outbound connection':'Information (NonCKC)',
'PUA-ADWARE Adware Goobzo/CrossRider variant outbound connection':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'PUA-ADWARE User-Agent adware OutBrowse/Amonitize':'Information (NonCKC)',
'PUA-ADWARE User-Agent adware OutBrowse/Amonitize':'Information (NonCKC)',
'PUA-ADWARE User-Agent adware OutBrowse/Amonitize':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'PUA-ADWARE InstallMetrix precheck stage outbound connection':'Information (NonCKC)',
'PUA-ADWARE InstallMetrix fetch offers stage outbound connection':'Information (NonCKC)',
'PUA-ADWARE InstallMetrix reporting binary installation stage status':'Information (NonCKC)',
'PUA-ADWARE InstallMetrix reporting fetch offers stage status':'Information (NonCKC)',
'PUA-ADWARE User-Agent Vitruvian':'Information (NonCKC)',
'PUA-ADWARE Vitruvian outbound connection':'Information (NonCKC)',
'PUA-ADWARE Vitruvian outbound connection':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'PUA-ADWARE SearchProtect user-agent detection':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Dyre publickey outbound connection':'Exploitation',
'PUA-ADWARE SuperOptimizer installation status':'Information (NonCKC)',
'PUA-ADWARE SuperOptimizer encrypted data transmission':'Information (NonCKC)',
'PUA-ADWARE SuperOptimizer geolocation request':'Information (NonCKC)',
'PUA-ADWARE Eorezo outbound connection':'Information (NonCKC)',
'PUA-ADWARE Eorezo get advertisement':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SERVER-WEBAPP vBulletin XSS redirect attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SERVER-WEBAPP Magento remote code execution attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'1,!&,0xF8,2':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'established,':'Information (NonCKC)',
'established,':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Metasploit Meterpreter reverse HTTPS certificate':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'PUA-ADWARE Win.Adware.Sendori user-agent detection':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.TorrentLocker/Teerac self-signed certificate':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Metasploit Meterpreter reverse HTTPS certificate':'Information (NonCKC)',
'INDICATOR-COMPROMISE Metasploit Meterpreter reverse HTTPS certificate':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Backdoor.Chopper web shell connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'PROTOCOL-DNS glibc getaddrinfo A record stack buffer overflow attempt':'Delivery',
'PROTOCOL-DNS glibc getaddrinfo AAAA record stack buffer overflow attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'POLICY-OTHER Polycom Botnet inbound connection attempt':'Delivery',
'POLICY-OTHER Polycom Botnet inbound connection attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.NetWiredRC variant connection setup':'Exploitation',
'MALWARE-CNC Win.Trojan.NetWiredRC variant failed read logs':'Exploitation',
'MALWARE-CNC Win.Trojan.NetWiredRC variant keepalive':'Exploitation',
'MALWARE-CNC Win.Trojan.NetWiredRC variant send credentials':'Exploitation',
'MALWARE-CNC Win.Trojan.NetWiredRC variant send mail credentials':'Exploitation',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.FTPKeyLogger outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'APP-DETECT Bloomberg web crawler outbound connection':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Qakbot variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.Godzilla downloader successful base64 binary download':'Exploitation',
'INDICATOR-COMPROMISE Content-Type text/plain containing Portable Executable data':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Trojan.NetWiredRC variant connection setup':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'MALWARE-CNC Win.Backdoor.NanoBot variant outbound connection':'Exploitation',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Content-Type image containing Portable Executable data':'Information (NonCKC)',
'SERVER-WEBAPP HttpOxy CGI application vulnerability potential man-in-the-middle attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'BROWSER-FIREFOX Mozilla Firefox about field spoofing attempt':'Delivery',
'EXPLOIT-KIT Phoenix Exploit Kit inbound geoip.php bdr exploit attempt':'Delivery',
'to_client':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'established,to_client':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'PUA-ADWARE MindSpark framework installer attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client':'Information (NonCKC)',
'to_client':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'PUA-ADWARE Sokuxuan outbound connection attempt':'Delivery',
'PUA-OTHER Bitcoin Mining subscribe Stratum protocol client request attempt':'Delivery',
'PUA-OTHER Bitcoin Mining authorize Stratum protocol client request attempt':'Delivery',
'PUA-OTHER Bitcoin Mining extranonce Stratum protocol subscribe client request attempt':'Delivery',
'PROTOCOL-OTHER TP-Link TDDP SET_CONFIG type buffer overflow attempt':'Delivery',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'PROTOCOL-OTHER TP-Link TDDP Get_config configuration leak attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SERVER-WEBAPP Western Digital MyCloud command injection attempt':'Delivery',
'SERVER-WEBAPP Western Digital MyCloud command injection attempt':'Delivery',
'SERVER-WEBAPP Western Digital MyCloud command injection attempt':'Delivery',
'SERVER-WEBAPP Western Digital MyCloud command injection attempt':'Delivery',
'MALWARE-CNC Win.Trojan.NetWiredRC variant registration message':'Exploitation',
'MALWARE-CNC Win.Trojan.NetWiredRC variant check logs':'Exploitation',
'MALWARE-CNC Win.Trojan.NetWiredRC variant keepalive':'Exploitation',
'POLICY-OTHER Cisco WebEx explicit use of web plugin':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SERVER-OTHER QNAP remote buffer overflow attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'INDICATOR-COMPROMISE Binary file download request from internationalized domain name using Microsoft BITS':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SERVER-WEBAPP DotNetNuke installation attempt detected':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'OS-WINDOWS Microsoft Windows empty RDP cookie negotiation attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'OS-WINDOWS Microsoft Windows SMB anonymous session IPC share access attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'established,':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'SERVER-WEBAPP /.svn/entries file access attempt':'Delivery',
'SERVER-WEBAPP /cgi-bin/sh file access attempt':'Delivery',
'SERVER-WEBAPP /etc/inetd.conf file access attempt':'Delivery',
'SERVER-WEBAPP /etc/motd file access attempt':'Delivery',
'SERVER-WEBAPP /etc/shadow file access attempt':'Delivery',
'SERVER-WEBAPP /ws_ftp.log file access attempt':'Delivery',
'SERVER-WEBAPP Oracle Application Server 9i unauthenticated application deployment attempt':'Delivery',
'POLICY-OTHER Teleopti WFM database information request detected':'Information (NonCKC)',
'POLICY-OTHER Teleopti WFM administrative user credentials request detected':'Information (NonCKC)',
'POLICY-OTHER Teleopti WFM administrative user creation detected':'Information (NonCKC)',
'SERVER-OTHER WSFTP IpSwitch custom SITE command execution attempt':'Delivery',
'SERVER-WEBAPP Kaspersky Linux File Server WMC cross site request forgery attempt':'Delivery',
'SERVER-WEBAPP Kaspersky Linux File Server WMC directory traversal attempt':'Delivery',
'SERVER-WEBAPP Kaspersky Linux File Server WMC directory traversal attempt':'Delivery',
'SERVER-WEBAPP Kaspersky Linux File Server WMC directory traversal attempt':'Delivery',
'SERVER-WEBAPP Kaspersky Linux File Server WMC cross site scripting attempt':'Delivery',
'MALWARE-CNC Osx.Trojan.Xagent outbound connection':'Exploitation',
'SERVER-WEBAPP Ubiquiti Networks UniFi Cloud Key Firm v0.6.1 Host Remote Command Execution attempt':'Delivery',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'POLICY-OTHER Cisco DDR2200 ASDL gateway file download detected':'Information (NonCKC)',
'SERVER-WEBAPP Cisco DDR2200 ADSL gateway command injection attempt':'Delivery',
'SERVER-WEBAPP Cisco DDR2200 ADSL gateway command injection attempt':'Delivery',
'SERVER-WEBAPP Cisco DDR2200 ADSL gateway command injection attempt':'Delivery',
'SERVER-WEBAPP Cisco DDR2200 ADSL gateway command injection attempt':'Delivery',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'MALWARE-CNC Potential hostile executable served from compromised or malicious WordPress site attempt':'Exploitation',
'MALWARE-CNC Potential hostile executable served from compromised or malicious WordPress site attempt':'Exploitation',
'to_client,established':'Information (NonCKC)',
'to_client,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'POLICY-OTHER NetSupport Manager RAT outbound connection detected':'Information (NonCKC)',
'established,to_server':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)',
'to_server,established':'Information (NonCKC)'
}



In [23]:
#Helper function created for aggregation of PCAP Files
connections=set() #the use of a set is for unique values (it appears)
#Found at : https://thepacketgeek.com/pyshark/packet-object/
data=[]
def parse(pkt):
    try:
        protocol =  pkt.transport_layer
        src_addr = pkt.ip.src
        src_port = pkt[pkt.transport_layer].srcport
        dst_addr = pkt.ip.dst
        dst_port = pkt[pkt.transport_layer].dstport
        time=pkt.sniff_time
        print ('%s,%s  %s:%s --> %s:%s' % (time,protocol, src_addr, src_port, dst_addr, dst_port))
        data.append(('%s,%s,%s:%s,%s:%s' % (time,protocol, src_addr, src_port, dst_addr, dst_port))) #append data to a list instead of printing
        #connections.add(('%s  %s:%s , %s:%s' % (protocol, src_addr, src_port, dst_addr, dst_port)))
    except AttributeError as e:
        #ignore packets that aren't TCP/UDP or IPv4
        pass

In [24]:
#********************************************************
#Step 4 Injest Alert JSON 

#Import all .json files within a directory
# merge them into one dataframe for analysis

#code from https://newbedev.com/python-read-several-json-files-from-a-folder
#********************************************************



#with open('dayonepresponse.json') as json_data:
#    data = json.load(json_data)

# this finds our json files
path_to_json = 'E:\\researchdata_replicationstudy\\NCCDC_logs-20150424\\dayone\\jsonresponsesfromsnort\\'

json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

print(json_files)

for index, js in enumerate(json_files):
    alerts = pd.DataFrame(columns=['time','alert'])
    with open(os.path.join(path_to_json, js)) as json_file:
        print("File Open: "+path_to_json+js)
        data = json.load(json_file)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
   

# now that we have the pertinent json data in our DataFrame let's look at it
#        print(jsons_data)

        df = pd.DataFrame(data['analyses'])
        df.dtypes
        #print(df.columns)
        #print(df["alerts"][0]['timestamp'])
        for each in data['analyses']:

            for each2 in each['alerts']:
                #print (each2['message'])
                #print (each2['timestamp'])
                alerts=alerts.append({'time':each2['timestamp'],'alert':each2['message']},ignore_index=True)
                   #df = df.append({'A': i}, ignore_index=True)
    print ("File Open: "+str(js))
    print ("ALerts Length: "+str(len(alerts)))
    print("Alerts Head:")
    print (alerts.head())
    
    alerts['time']=pd.to_datetime(alerts['time']) #Convert Alert Colum into A DateTime
    print("Alerts Head (After To_DateTime):")
    print (alerts.head())
    alerts.dtypes
    pcapcsvfile="E:\\researchdata_replicationstudy\\NCCDC_logs-20150424\\dayone\\pcaps\\"+str(js).replace(".pcap-pcapresponse.json","")+".csv"
    print("File to be Read:" + pcapcsvfile)
    dfpcap=pd.read_csv(pcapcsvfile, encoding='utf-16', error_bad_lines=False) 
    
    print("DFPCAP Shape: "+str(dfpcap.shape))
    print("DFPCAP Head: ")
    print(dfpcap.head())
    
    dfpcap=dfpcap.rename(columns={"_ws.col.Time": "time"}) #Convert Dataframe containing all PCAP data Colum TIme into a Date Time
    dfpcap=dfpcap.rename(columns={"_ws.col.Protocol": "protocol"}) #Convert Dataframe containing all PCAP data Colum TIme into a Date Time
    dfpcap=dfpcap.rename(columns={"ip.src": "source"}) #Convert Dataframe containing all PCAP data Colum TIme into a Date Time
    dfpcap=dfpcap.rename(columns={"ip.dst": "destination"}) #Convert Dataframe containing all PCAP data Colum TIme into a Date Time
    dfpcap['time']=pd.to_datetime(dfpcap['time'])
    dfpcap.dtypes
    print("DFPCAP After Column Rename:")
    print(dfpcap.head())
    
    #result=pd.merge(dfpcap,alerts, on='time', how='left') #Merge the alert dataframe and the PCAP Data frame into 1
    result=pd.merge(dfpcap,alerts, on='time', how='left')#Z, suffixes=('_1', '_2'))#,coerce=True)
    print("Result Shape: "+str(result.shape))
    print("Result Head")
    print(result.head())
    print("Results Group By Alert, Protocol & Source")
    print(str(result.groupby(["alert","protocol","source"])["protocol"].count()))
    #print(result)
    print("Alerts that are Not NA:")
    print(result.loc[~result['alert'].isna()])
    
    print("Alerts that are  NA:")
    print(result.loc[result['alert'].isna()])
        #Confirm merged results
    with pd.option_context('display.max_rows', 500, 'display.max_columns', None,'display.width', 1000):  # more options can be specified also
       # print(df)
    #print(result.columns)
        #print(result.loc[df['alert'] != NaN])
        print(result.loc[~result['alert'].isna()])
        #print(result)
        
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(result.groupby(["alert","protocol","source"])["protocol"].count())
        
    dot=Digraph()
    dot.format = 'svg'
    dot=dot.attr(width="900pt", height="1800pt",fixedsize='true')
    temp=result.loc[~result['alert'].isna()]

    temp.head()
    #temp=temp.sample(50)
    temp=temp.drop_duplicates(subset = ["source","destination","alert"])
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #print(df)
    uniquedest=temp.groupby("destination",sort=True).count().index
    print(str(result.groupby("destination",sort=True).count()))
    
    pcapfile=str(js).replace(".pcap-pcapresponse.json","")
    print(pcapfile)
    
    print(uniquedest)

    for dest in uniquedest:
        dot = Digraph()
        t2=temp.loc[temp['destination']==dest]
        for index, row in t2.iterrows():
                ckcvalue=ckc_mapping.get(str(row["alert"]),"Unknown CKC")
                dot.edge(str(row["source"]).replace(":",'.'), str(row["destination"]).replace(":",'.'), label=str(row["alert"])+"("+ckcvalue+")")
                dot.node_attr.update(color='red',layout='twopi')
        dot.render('E:\\researchdata_replicationstudy\\NCCDC_logs-20150424\\dayone\\pdfs\\'+dest.replace(".","_")+"-"+str(pcapfile))#,view=True)

['dayone.0000.pcap-pcapresponse.json', 'dayone.0001.pcap-pcapresponse.json', 'dayone.0002.pcap-pcapresponse.json', 'dayone.0003.pcap-pcapresponse.json', 'dayone.0004.pcap-pcapresponse.json', 'dayone.0005.pcap-pcapresponse.json', 'dayone.0006.pcap-pcapresponse.json', 'dayone.0007.pcap-pcapresponse.json', 'dayone.0008.pcap-pcapresponse.json', 'dayone.0009.pcap-pcapresponse.json', 'dayone.0010.pcap-pcapresponse.json', 'dayone.0011.pcap-pcapresponse.json', 'dayone.0012.pcap-pcapresponse.json', 'dayone.0013.pcap-pcapresponse.json', 'dayone.0014.pcap-pcapresponse.json', 'dayone.0015.pcap-pcapresponse.json', 'dayone.0016.pcap-pcapresponse.json', 'dayone.0017.pcap-pcapresponse.json', 'dayone.0018.pcap-pcapresponse.json', 'dayone.0019.pcap-pcapresponse.json', 'dayone.0020.pcap-pcapresponse.json', 'dayone.0021.pcap-pcapresponse.json', 'dayone.0022.pcap-pcapresponse.json', 'dayone.0023.pcap-pcapresponse.json', 'dayone.0024.pcap-pcapresponse.json', 'dayone.0025.pcap-pcapresponse.json', 'dayone.002

b'Skipping line 63: expected 7 fields, saw 9\nSkipping line 294: expected 7 fields, saw 9\nSkipping line 295: expected 7 fields, saw 9\nSkipping line 2475: expected 7 fields, saw 9\nSkipping line 2476: expected 7 fields, saw 9\nSkipping line 3799: expected 7 fields, saw 9\nSkipping line 3806: expected 7 fields, saw 9\nSkipping line 7058: expected 7 fields, saw 9\nSkipping line 7059: expected 7 fields, saw 9\nSkipping line 7814: expected 7 fields, saw 9\nSkipping line 9027: expected 7 fields, saw 9\nSkipping line 9030: expected 7 fields, saw 9\nSkipping line 10725: expected 7 fields, saw 9\nSkipping line 10737: expected 7 fields, saw 9\nSkipping line 10805: expected 7 fields, saw 9\nSkipping line 12581: expected 7 fields, saw 9\nSkipping line 12590: expected 7 fields, saw 9\nSkipping line 15990: expected 7 fields, saw 9\nSkipping line 16468: expected 7 fields, saw 9\nSkipping line 16479: expected 7 fields, saw 9\nSkipping line 16716: expected 7 fields, saw 9\nSkipping line 18537: expect

b'Skipping line 1311013: expected 7 fields, saw 9\nSkipping line 1313541: expected 7 fields, saw 9\nSkipping line 1313651: expected 7 fields, saw 9\nSkipping line 1313656: expected 7 fields, saw 9\nSkipping line 1317405: expected 7 fields, saw 9\nSkipping line 1318728: expected 7 fields, saw 9\nSkipping line 1318765: expected 7 fields, saw 9\nSkipping line 1318789: expected 7 fields, saw 9\nSkipping line 1318895: expected 7 fields, saw 9\nSkipping line 1321098: expected 7 fields, saw 9\nSkipping line 1321843: expected 7 fields, saw 9\nSkipping line 1324835: expected 7 fields, saw 9\nSkipping line 1324920: expected 7 fields, saw 9\nSkipping line 1324962: expected 7 fields, saw 9\nSkipping line 1331075: expected 7 fields, saw 9\nSkipping line 1331277: expected 7 fields, saw 9\nSkipping line 1331278: expected 7 fields, saw 9\nSkipping line 1332340: expected 7 fields, saw 9\nSkipping line 1334213: expected 7 fields, saw 9\nSkipping line 1334216: expected 7 fields, saw 9\nSkipping line 1334

DFPCAP Shape: (1351133, 7)
DFPCAP Head: 
                   _ws.col.Time frame.number            ip.src  \
0  '2015-04-23 14:51:30.891359'          '1'      '50.7.55.82'   
1  '2015-04-23 14:51:30.891373'          '2'      '50.7.55.82'   
2  '2015-04-23 14:51:30.891947'          '3'  '54.228.242.170'   
3  '2015-04-23 14:51:30.892106'          '4'   '10.121.200.16'   
4  '2015-04-23 14:51:30.892166'          '5'   '10.121.200.15'   

             ip.dst tcp.dstport udp.dstport _ws.col.Protocol  
0   '10.121.200.16'     '42403'         NaN            'TCP'  
1   '10.121.200.16'     '42403'         NaN            'TCP'  
2   '10.121.200.15'     '57778'         NaN            'TCP'  
3      '50.7.55.82'        '80'         NaN            'TCP'  
4  '54.228.242.170'       '443'         NaN        'TLSv1.2'  
DFPCAP After Column Rename:
                        time frame.number            source       destination  \
0 2015-04-23 14:51:30.891359          '1'      '50.7.55.82'   '10.121.200.1

alert                                                                   protocol   source           
BAD-TRAFFIC same SRC/DST                                                'DHCP'     '0.0.0.0'             124
COMMUNITY WEB-MISC mod_jrun overflow attempt                            'HTTP'     '10.121.200.39'         4
                                                                        'TCP'      '192.168.2.4'          86
DNS zone transfer TCP                                                   'DNS'      '192.168.9.2'           1
ICMP Echo Reply                                                         'ICMP'     '10.10.10.10'           3
                                                                                   '10.10.10.108'          2
                                                                                   '10.10.10.109'          2
                                                                                   '10.10.10.15'           3
                           

...

alert                                                                        protocol   source          
ATTACK-RESPONSES id check returned userid                                    'TCP'      '10.80.80.10'         1
BAD-TRAFFIC same SRC/DST                                                     'DHCP'     '0.0.0.0'             6
COMMUNITY WEB-MISC Hasbani-WindWeb GET DoS attempt                           'HTTP'     '10.132.86.26'        1
COMMUNITY WEB-MISC Proxy Server Access                                       'TCP'      '10.120.0.200'       48
COMMUNITY WEB-MISC Test Script Access                                        'HTTP'     '10.132.86.26'        5
COMMUNITY WEB-MISC mod_jrun overflow attempt                                 'TCP'      '10.110.0.223'        5
COMMUNITY WEB-PHP XSS attempt                                                'HTTP'     '10.132.86.26'        1
                                                                             'TCP'      '10.132.86.26'        1

File Open: E:\researchdata_replicationstudy\NCCDC_logs-20150424\dayone\jsonresponsesfromsnort\dayone.0043.pcap-pcapresponse.json
File Open: dayone.0043.pcap-pcapresponse.json
ALerts Length: 4106
Alerts Head:
                         time                                alert
0  2015-04-23T15:24:35.730499                 WEB-MISC /etc/passwd
1  2015-04-23T15:24:35.730499          WEB-MISC mylog.phtml access
2  2015-04-23T15:24:35.775280                 WEB-MISC /etc/passwd
3  2015-04-23T15:24:35.775051            WEB-IIS iissamples access
4  2015-04-23T15:24:35.814854  WEB-COLDFUSION administrator access
Alerts Head (After To_DateTime):
                        time                                alert
0 2015-04-23 15:24:35.730499                 WEB-MISC /etc/passwd
1 2015-04-23 15:24:35.730499          WEB-MISC mylog.phtml access
2 2015-04-23 15:24:35.775280                 WEB-MISC /etc/passwd
3 2015-04-23 15:24:35.775051            WEB-IIS iissamples access
4 2015-04-23 15:24:35.81485

b'Skipping line 20754: expected 7 fields, saw 9\nSkipping line 24504: expected 7 fields, saw 9\nSkipping line 24612: expected 7 fields, saw 9\nSkipping line 70548: expected 7 fields, saw 9\nSkipping line 76274: expected 7 fields, saw 9\nSkipping line 94676: expected 7 fields, saw 9\n'
b'Skipping line 147381: expected 7 fields, saw 9\nSkipping line 147682: expected 7 fields, saw 9\nSkipping line 220487: expected 7 fields, saw 9\nSkipping line 220602: expected 7 fields, saw 9\nSkipping line 259575: expected 7 fields, saw 9\nSkipping line 259605: expected 7 fields, saw 9\n'
b'Skipping line 279556: expected 7 fields, saw 9\nSkipping line 314818: expected 7 fields, saw 9\nSkipping line 314819: expected 7 fields, saw 9\nSkipping line 348987: expected 7 fields, saw 9\nSkipping line 367589: expected 7 fields, saw 9\nSkipping line 383295: expected 7 fields, saw 9\n'
b'Skipping line 401536: expected 7 fields, saw 9\nSkipping line 401601: expected 7 fields, saw 9\nSkipping line 405854: expected 7

DFPCAP Shape: (646516, 7)
DFPCAP Head: 
                   _ws.col.Time frame.number            ip.src  \
0  '2015-04-23 15:24:35.726777'          '1'    '10.120.0.200'   
1  '2015-04-23 15:24:35.726779'          '2'   '192.168.50.99'   
2  '2015-04-23 15:24:35.726780'          '3'   '192.168.50.99'   
3  '2015-04-23 15:24:35.726782'          '4'    '10.10.10.103'   
4  '2015-04-23 15:24:35.726783'          '5'  '192.168.250.80'   

           ip.dst tcp.dstport udp.dstport _ws.col.Protocol  
0  '10.10.10.103'     '50361'         NaN            'TCP'  
1   '10.120.0.30'        '80'         NaN            'TCP'  
2   '10.120.0.30'        '80'         NaN            'TCP'  
3  '10.120.0.200'      '8080'         NaN            'TCP'  
4  '10.132.86.26'     '49532'         NaN            'TCP'  
DFPCAP After Column Rename:
                        time frame.number            source     destination  \
0 2015-04-23 15:24:35.726777          '1'    '10.120.0.200'  '10.10.10.103'   
1 2015-04-2

alert                                                                        protocol   source           
BACKDOOR CDK                                                                 'FINGER'   '192.168.233.76'       1
BAD-TRAFFIC same SRC/DST                                                     'DHCP'     '0.0.0.0'              9
BAD-TRAFFIC tcp port 0 traffic                                               'TCP'      '10.80.80.205'         1
                                                                                        '192.168.233.76'       1
COMMUNITY WEB-MISC Hasbani-WindWeb GET DoS attempt                           'HTTP'     '10.132.86.26'         9
COMMUNITY WEB-MISC Proxy Server Access                                       'HTTP'     '10.120.0.200'         1
                                                                             'TCP'      '10.120.0.200'         2
COMMUNITY WEB-MISC Test Script Access                                        'HTTP'     '10.132.86.26' 

 ***How to Export a UTF-16 CSV version of a PCAP using Wireshark's TShark***
 
 ts -r .\pcaps\dayone.0044.pcap -t ad -T fields -e _ws.col.Time -e frame.number -e ip.src -e ip.dst -e tcp.dstport -e udp.dstport -e _ws.col.Protocol -E separator="," -E quote=s -E header=y > .\pcaps\testZ4.txt

In [25]:
#import csv
#Review this later, seems like there are atleast several errors in the data
#https://ask.wireshark.org/question/16830/tshark-frametime-format/
#    dfpcap=pd.read_csv("E:\\researchdata_replicationstudy\\NCCDC_logs-20150424\\dayone\\pcaps\\testZ4.txt", encoding='utf-16', error_bad_lines=False)#,error_bad_lines=False)#,skiprows=1)#,nrows=12,
    

In [26]:
    #********************************************************
    #Step5 Injest PCAP Files
    #still within the loop for a single alert, injest the accompanying PCAP


    #Import all .pcap files within a directory
    # merge them into one dataframe for analysis


    #********************************************************


#    path_to_pcapsresponses = 'E:\\researchdata_replicationstudy\\NCCDC_logs-20150424\\dayone\\pcaps\\'

    
#    cap=[]

#    print(str(path_to_pcapsresponses+js))
#    with open(str(path_to_pcapsresponses+js).replace("-pcapresponse.json","")) as pcap_file:
      
#        cap = pyshark.FileCapture(pcap_file)#,only_summaries=True)
#        for pkt in cap:
#            parse(pkt)  #for each packet in the capture go through and append certain data to a list




In [27]:
 #   cap2=cap #copy set to avoid having to reimport!
 #   tmp=list(cap2)
 #   dfcap=pd.DataFrame(tmp)
 #   dfcap.head()

In [28]:
#    dfpcap=dfpcap.rename(columns={"Time": "time"}) #Convert Dataframe containing all PCAP data Colum TIme into a Date Time
#    dfpcap